# Dissecting PubMed
## Which content is covered by the Library? and Open Access?
#### Floriane Muller & Pablo Iriarte, Geneva University Library, Switzerland


## Merge PubMed metadata with UNIGE Library journals coverage

### Merging with print journals holdings
These holdings where exported on 20.09.2017 by Benoît Maurice, system librarian for the Geneva RERO network, from the ILS Virtua via SQL connection. It includes all the holdings of journals with ISSN for the localizations of Science and Medicine. The export was repeated in 05.12.2017 to include the DBU central archive of the Geneva University Library containing journals archives of Medicine and Sciences sections.

The SQL query :
```
select holdingsid, subfield_data, bibid, location, iss.issn_number
from iso_2709  iso, holdlink lin, issn iss
where idtype = 104
and field_tag = '866'
and subfield_code = 'a'
and iso.id = lin.holdingsid
and lin.location like '610200%' # Locations
and lin.bibid = iss.bib_id
```
Repeated for locations :
* 6102000%	geucmu	GE UC MEDECINE ---
* 6102100%	geucds	GE UC CDS ---
* 6102700%	gemchi	GE HUG Chirurgie ---
* 6102900%	geuhim	GE UC IEH2 ---
* 6101100%	geussm	GE UA MATH ---	GE Uni Arve - Mathématiques (Bibl. Georges de Rham)
* 6101200%	geusob	GE UA ASTRO ---	GE Uni Arve - Astronomie (Observatoire)
* 6101400%	geussc	GE UA BELS ---	GE Uni Arve - Anthropologie, Biologie, Chimie, Physique (BELS)
* 6101500%	geusst	GE UA TERRE ---	GE Uni Arve - Sciences de la Terre et de l'environnement
* 6101600%	geussa	GE UA ANTHRO ---	GE Uni Arve - Anthropologie
* 6109600%	geudbu	GE UX DBU ---	GE Uni CODIS - Dépôt de la Bibliothèque de l'Université

The CSV files were fusioned with bash (unix shell on Cygwin) :
```bash
cat *.tab > export_print_journals_20171205.csv
```

The final csv was cleaned with Notepad++:
1. Remove headers : find "#HOLDINGSID" and remove lines manually
2. Remove carriage returns introduced accidentally by cataloguers in holdings text field :
    * replace ```^([^\t]+\t[^\t]+)\r\n by \1```
    * replace ```^([^\t]+\t[^\t]+)\n by \1```
    * replace ```^([^\t]+\t[^\t]+)\r by \1```
3. Tag lines well formed
    * replace ```^([^\t]+\t[^\t]+\t[^\t]+\t[^\t]+\t[^\t]+)$ by £££\1```
    * find and manually correct bad lines: ```^[^£]```
    * count lines ```^([^\t]+\t[^\t]+\t[^\t]+\t[^\t]+\t[^\t]+)$``` to check if all are weel formed
4. Replace "#HOLDINGSID" as "HOLDINGSID" in the first line
5. Save cleaned file as 'data/sources/library/pjournals/export_print_journals_20170920_cleaned.csv'

In [5]:
# display the full content of rows (non truncated)
import pandas as pd
pd.set_option('display.max_colwidth', -1)

# Extract ISSN and year from print journals holdings cataloguing data
journals_brut = pd.read_csv('data/sources/library/pjournals/export_print_journals_20171205_cleaned.csv', delimiter='\t', header=0, usecols=['ISSN_NUMBER', 'SUBFIELD_DATA', 'LOCATION'])
journals_brut

,SUBFIELD_DATA,LOCATION,ISSN_NUMBER
0,Vol. 55(1966) - 65(1976),610270001,0003-9772
1,Vol. 17(1985)- 40(2006),610270001,0041-1345
2,Vol. 16(1973)-40(1995),610270001,0071-7916
3,Vol. 1(1961)-12(1973); 14(1974),610270001,0079-6824
4,Vol. 8(1984)-19(1994) icpl; 20(1995)-37(2012),610270001,0266-7681
5,65-160 (1964-1976) icpl; 167 (1977)-,610270001,0300-8827
6,Vol. 7(1982)-34(2009),610270001,0362-2436
7,Vol. 2(1978)-36(2012),610270001,0364-2313
8,Vol. 1(1982) - 4(1989),610270001,0753-9053
9,Vol 2 (1987)-38(2009),610270001,0885-3177


In [6]:
# Exclude collection from "not available" location : 610140016
journals = journals_brut[journals_brut.LOCATION != 610140016]
journals

,SUBFIELD_DATA,LOCATION,ISSN_NUMBER
0,Vol. 55(1966) - 65(1976),610270001,0003-9772
1,Vol. 17(1985)- 40(2006),610270001,0041-1345
2,Vol. 16(1973)-40(1995),610270001,0071-7916
3,Vol. 1(1961)-12(1973); 14(1974),610270001,0079-6824
4,Vol. 8(1984)-19(1994) icpl; 20(1995)-37(2012),610270001,0266-7681
5,65-160 (1964-1976) icpl; 167 (1977)-,610270001,0300-8827
6,Vol. 7(1982)-34(2009),610270001,0362-2436
7,Vol. 2(1978)-36(2012),610270001,0364-2313
8,Vol. 1(1982) - 4(1989),610270001,0753-9053
9,Vol 2 (1987)-38(2009),610270001,0885-3177


In [7]:
# Split lines with ";"
journals = pd.concat([pd.Series(row['ISSN_NUMBER'], row['SUBFIELD_DATA'].split(';'))
                      for _, row in journals.iterrows()]).reset_index()
journals.columns = ['SUBFIELD_DATA_BRUT', 'ISSN_NUMBER']
journals

,SUBFIELD_DATA_BRUT,ISSN_NUMBER
0,Vol. 55(1966) - 65(1976),0003-9772
1,Vol. 17(1985)- 40(2006),0041-1345
2,Vol. 16(1973)-40(1995),0071-7916
3,Vol. 1(1961)-12(1973),0079-6824
4,14(1974),0079-6824
5,Vol. 8(1984)-19(1994) icpl,0266-7681
6,20(1995)-37(2012),0266-7681
7,65-160 (1964-1976) icpl,0300-8827
8,167 (1977)-,0300-8827
9,Vol. 7(1982)-34(2009),0362-2436


In [8]:
# Remove pages ex. p.1112-1195
pat_pages = (r'p\.[0-9\- ]+')
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA_BRUT'].str.replace(pat_pages, '', case=False)

In [9]:
# See volumes with 4 digts that can be dectected as years
pat_vol_year = (r'(?P<one>\d{4})\((?P<two>[0-9\/]+)\)')
journals[journals.SUBFIELD_DATA_BRUT.str.contains(pat_vol_year)]

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
1477,1598(2002)-1774(2007),1570-9639,1598(2002)-1774(2007)
1966,"ANNO I, VOL.1,NO1(1984)-N0235(2005)",0393-2095,"ANNO I, VOL.1,NO1(1984)-N0235(2005)"
2048,1436(1998/99)-1771(2007),0005-2760,1436(1998/99)-1771(2007)
2390,"Vol. 282, No 1820(2012)-283, No 1845(2016)",0962-8452,"Vol. 282, No 1820(2012)-283, No 1845(2016)"
2391,"Vol. 279, No 1729(2012)-282, No 1818(2015)",0962-8452,"Vol. 279, No 1729(2012)-282, No 1818(2015)"
2392,"Vol. 278, No 1702(2011)-279, No 1727(2012)",0962-8452,"Vol. 278, No 1702(2011)-279, No 1727(2012)"
2437,1(1966)-1191(2008),0006-8993,1(1966)-1191(2008)
5130,78(1964/65)-1094(1978) icpl.,0032-8707,78(1964/65)-1094(1978) icpl.
5443,Fasc. 2181(1992),0029-6147,Fasc. 2181(1992)
5444,Fasc. 2178(1991),0029-6147,Fasc. 2178(1991)


In [12]:
# Remove volumes with 4 digts that can be dectected as years
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(r'(?P<one>\d{4})\((?P<two>[0-9\/]+)\)', r'DDDD(\2)')
journals[journals.SUBFIELD_DATA_BRUT.str.contains(pat_vol_year)]

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
1477,1598(2002)-1774(2007),1570-9639,DDDD(2002)-DDDD(2007)
1966,"ANNO I, VOL.1,NO1(1984)-N0235(2005)",0393-2095,"ANNO I, VOL.1,NO1(1984)-NDDDD(2005)"
2048,1436(1998/99)-1771(2007),0005-2760,DDDD(1998/99)-DDDD(2007)
2390,"Vol. 282, No 1820(2012)-283, No 1845(2016)",0962-8452,"Vol. 282, No DDDD(2012)-283, No DDDD(2016)"
2391,"Vol. 279, No 1729(2012)-282, No 1818(2015)",0962-8452,"Vol. 279, No DDDD(2012)-282, No DDDD(2015)"
2392,"Vol. 278, No 1702(2011)-279, No 1727(2012)",0962-8452,"Vol. 278, No DDDD(2011)-279, No DDDD(2012)"
2437,1(1966)-1191(2008),0006-8993,1(1966)-DDDD(2008)
5130,78(1964/65)-1094(1978) icpl.,0032-8707,78(1964/65)-DDDD(1978) icpl.
5443,Fasc. 2181(1992),0029-6147,Fasc. DDDD(1992)
5444,Fasc. 2178(1991),0029-6147,Fasc. DDDD(1991)


In [13]:
# Looking for missing indications
journals[journals.SUBFIELD_DATA_BRUT.str.contains('manq')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
471,"19(1994), n° 1 manquant",0363-9762,"19(1994), n° 1 manquant"
745,"manque Vol 8,no 1(2008)",1567-133X,"manque Vol 8,no 1(2008)"
1544,"[manque vol 501 n°7465, sept. 2013]",0028-0836,"[manque vol 501 n°7465, sept. 2013]"
1545,"[manque vol. 489 n°7416, sept. 2012]",0028-0836,"[manque vol. 489 n°7416, sept. 2012]"
1546,"[manque vol 462 n°7574, déc. 2009]",0028-0836,"[manque vol 462 n°7574, déc. 2009]"
1920,"85(2004), 6 - 12 manquant",0221-0363,"85(2004), 6 - 12 manquant"
1961,"33(2008), manque n°7",0377-8282,"33(2008), manque n°7"
2029,[numéro manquant:],0002-838X,[numéro manquant:]
2032,manque: Vol 39 n° 1,0002-8614,manque: Vol 39 n° 1
2481,97(1953) manquant,0039-6087,97(1953) manquant


In [14]:
# Remove all date after "- manq*" OR "manq*" OR "mq* OR "lacune" or "index" 
pat1 = (r'[-] manq.*')
pat2 = (r'manq.*')
pat3 = (r'mq.*')
pat4 = (r'[\|\[\(]lacun[^\|\[\(]+[\|\]\)]')
pat5 = (r'lacune[^-]+')
pat6 = (r'index.*')
pat7 = (r'indice.*')

# Remove identified recurrent superfluous "-" that could be percieved as open collection when in truth it is not

# Remove all "month.-month." and "month.- month." as they may be taken for an open collection.
# NOT WORKING pat9 = (r'[-][fév|mar|avr|mai|juin|juil|aoû|sept|oct|nov|déc|feb|apr|may|jun|jul|aug|dec]')
pat8 = (r'[a-z|A-Z|é|û]{1,9}\.-[a-z|A-Z|é|û]{1,9}\.')
pat9 = (r'[a-z|A-Z|é|û]{1,9}\.- [a-z|A-Z|é|û]{1,9}\.')

# Remove some specific artifacts: Words with "-" as they may be taken for an open collection :
# Belle-Idée, CD-ROM, Hors-Série, fasc. 7a-b, part I-II, titre-table, year-end, gas-greenhouse, sud-ouest, etc.
# Some words have "- " like "Ur- Fruehgesch"
# Be carefull to not remove "-vol." or "- vol."
pat10 = (r'[a-záéíóúàèìòùäëïöüâêîôû]{1,}[-][a-záéíóúàèìòùäëïöüâêîôû]{1,}')
pat11 = (r'[a-záéíóúàèìòùäëïöüâêîôû]{1,}[-] [a-záéíóúàèìòùäëïöüâêîôû]{1,}')

# Remove some specific artifacts:
# - don DSPM
pat12 = (r'[-] don DSMP.*')
# - Supplement
pat13 = (r'[-] suppl.*')
# cote: RA 1815
pat14 = (r'cote: [a-zA-Z0-9\-\.\/]+ \d{4}')
# cote RA 1777
pat15 = (r'cote [a-zA-Z0-9\-\.\/]+ \d{4}')
# John von Neumann, 1903-1957
pat16 = (r'John von Neumann, 1903-1957')

# Apply the patterns to remove strings (case insensitive)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat1, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat2, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat3, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat4, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat5, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat6, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat7, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat8, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat9, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat10, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat11, '', case=False) 
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat12, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat13, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat14, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat15, '', case=False)
journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA'].str.replace(pat16, '', case=False)


In [15]:
journals[journals.SUBFIELD_DATA_BRUT.str.contains('manq')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
471,"19(1994), n° 1 manquant",0363-9762,"19(1994), n° 1"
745,"manque Vol 8,no 1(2008)",1567-133X,
1544,"[manque vol 501 n°7465, sept. 2013]",0028-0836,[
1545,"[manque vol. 489 n°7416, sept. 2012]",0028-0836,[
1546,"[manque vol 462 n°7574, déc. 2009]",0028-0836,[
1920,"85(2004), 6 - 12 manquant",0221-0363,"85(2004), 6 - 12"
1961,"33(2008), manque n°7",0377-8282,"33(2008),"
2029,[numéro manquant:],0002-838X,[numéro
2032,manque: Vol 39 n° 1,0002-8614,
2481,97(1953) manquant,0039-6087,97(1953)


In [16]:
journals[journals.SUBFIELD_DATA_BRUT.str.contains('lacune')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
640,"1(1978)-21(1998) [lacunes: no. 3, vol. 16(1993)]",0149-5992,1(1978)-21(1998) [
848,1(1967)-16(1986) [atttention lacunes],0075-4331,1(1967)-16(1986) [atttention
1410,"139(2004)-NO1, 175(2008) [lacunes]",0379-0738,"139(2004)-NO1, 175(2008)"
1919,83(2002) lacunes (p.1112-1195),0221-0363,83(2002)
2593,180(2008)[lacunes],0936-6652,180(2008)
2780,[Nombreuses lacunes.],0073-0033,[Nombreuses
3326,"Vol. 10(2006)[lacunes] - Vol. 17, No. 1(2013)",1267-8694,"Vol. 10(2006) - Vol. 17, No. 1(2013)"
3539,"37(1995)-88(2007) lacunes pour les vol. 43(1997), 51(1999), 76(2004) et 88(2007)",0006-3525,37(1995)-88(2007)
3838,1(1973)-27(1999) [lacunes],0091-634X,1(1973)-27(1999)
4010,[Très nombreuses lacunes.],0003-8938,[Très nombreuses


In [17]:
journals[journals.SUBFIELD_DATA_BRUT.str.contains('index')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
4729,Manquent index annuels 2006-2008,0025-5629,
4933,[annual subject index 25(1951)-30(1956)],0006-3169,[annual subject
5856,index 1958-1959,0001-6705,
6466,Jusqu'à 1975 index des vol. seulement,0039-0518,Jusqu'à 1975
6552,[+ index 1(1958)-15(1966),0006-2952,[+
7039,"Mq Subject index 1996, part 1",0036-8091,
7219,[35(1980) index uniquement.],0007-5302,[35(1980)
9550,+ index des vol. 41(1995)-80(2000),0165-1684,+
9769,Manque vol. 11(1985) subject index,0098-9819,
9771,"Mq Aut. index 2000, part. 1",0098-9819,


In [18]:
journals[journals.SUBFIELD_DATA_BRUT.str.contains('fasc.')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
50,Vol. 3 fasc.4 (2003)-11(2011),1628-8319,Vol. 3 fasc.4 (2003)-11(2011)
197,Vol. 1 (1986)-20(fasc. 1-4) (2006),0299-2213,Vol. 1 (1986)-20(fasc. 1-4) (2006)
205,Vol. 91(1978)-125 fasc. 5(2005),0936-8051,Vol. 91(1978)-125 fasc. 5(2005)
270,Vol. 6 fasc. 4(2003)-13(2008),1098-3511,Vol. 6 fasc. 4(2003)-13(2008)
2041,T. 14 fasc. 1-4(1935),0004-1947,T. 14 fasc. 1-4(1935)
2042,t. 15 fasc. 1-4(1936),0004-1947,t. 15 fasc. 1-4(1936)
2043,"t. 16 fasc. 1, 3(1937)",0004-1947,"t. 16 fasc. 1, 3(1937)"
2044,"t. 17, fasc. 1-3(1937)",0004-1947,"t. 17, fasc. 1-3(1937)"
2045,"t. 18, fasc. 1-3(1938)",0004-1947,"t. 18, fasc. 1-3(1938)"
2046,"t. 19, fasc. 1-2(1938)",0004-1947,"t. 19, fasc. 1-2(1938)"


In [19]:
journals[journals.SUBFIELD_DATA_BRUT.str.contains('table')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
5586,[+ 4 vol. contenant les tables des matières 1956-1961.],0007-7135,[+ 4 vol. contenant les tables des matières 1956-1961.]
5996,Consultable en version électronique,0022-4359,Consultable en version électronique
6869,"Vol. 98 (1956), fasc. juill., août., oct.-déc. + titre-table",0007-6287,"Vol. 98 (1956), fasc. juill., août., +"
6872,"Vol. 138 (1996), fasc. mars-juin + titre-table",0007-6287,"Vol. 138 (1996), fasc. +"
6876,"Vol. 133 (1991), fasc. août-déc. + titre-table",0007-6287,"Vol. 133 (1991), fasc. . +"
6881,"Vol. 147 (2005), fasc. janv.-avril, juin + titre-table",0007-6287,"Vol. 147 (2005), fasc. janv.-avril, juin +"
11951,Consultable en version électronique,0378-8733,Consultable en version électronique
14046,[1-42 tables des matières seulement.],0256-9450,[1-42 tables des matières seulement.]
14656,+ table des matières des n. 1-50 = 1979-1999,1420-0252,+ table des matières des n. 1-50 = 1979-1999
15340,Consultable en version électronique,1042-9573,Consultable en version électronique


In [20]:
journals[journals.SUBFIELD_DATA_BRUT.str.contains('dic.')]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
3749,pour les volumes précédents voir Archives of Internal Medicine,2168-6106,pour les volumes précédents voir Archives of Internal Medicine
4748,"Manque : Serie 5, n. 36, ottobre-dicembre 2006 (pas reçu)",0029-6295,
4751,"N.S., n. 31(gennaio-marzo 1973) - n. 56(ottobre-dicembre 1977). Serie 3 : n. 1(gennaio-marzo 1982) - n. 37 (gennaio-marzo 1991). Serie 4 : n. 2(gennaio-marzo 1995) - n. 5(ottobre-dicembre 1995)",0029-6295,"N.S., n. 31( 1973) - n. 56( 1977). Serie 3 : n. 1( 1982) - n. 37 ( 1991). Serie 4 : n. 2( 1995) - n. 5( 1995)"
4752,n. 7(aprile-giugno 1996) - n. 12(luglio-dicembre 1997). Serie 5 : n. 1/2(gennaio-giugno 1998) - 63(2013),0029-6295,n. 7( 1996) - n. 12( 1997). Serie 5 : n. 1/2( 1998) - 63(2013)
7473,89(octubre-diciembre 1974)-95(abril-junio 1976),0034-9631,89( 1974)-95( 1976)
7927,"Les ""Supplementi"" sont à inscrire séparément sous ""Supplemento ai Rendiconti del circolo...""",0009-725X,"Les ""Supplementi"" sont à inscrire séparément sous ""Supplemento ai Rendiconti del circolo..."""
9581,[Indices de voces: anejo 88.],0210-9174,[
12085,[certains numéros reliés avec les volumes de Psychological medicine (journal)],0264-1801,[certains numéros reliés avec les volumes de Psychological medicine (journal)]
12277,"anno 27, n. 2 (luglio-dicembre 1986)",0516-6551,"anno 27, n. 2 ( 1986)"
13479,1(mayo 1988)-16(diciembre 1999),0214-4581,1(mayo 1988)-16(diciembre 1999)


In [21]:
# Test some ISSN
journals[journals.ISSN_NUMBER == '0937-9819']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
18059,21(2011)-25(2015),0937-9819,21(2011)-25(2015)
18060,10(2000)-15(2005),0937-9819,10(2000)-15(2005)
18061,17(2007)-18(2008) [lacunes],0937-9819,17(2007)-18(2008)
18062,1(1991)-7(1996/97),0937-9819,1(1991)-7(1996/97)


In [22]:
journals[journals.ISSN_NUMBER == '1267-8694']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
3325,-,1267-8694,-
3326,"Vol. 10(2006)[lacunes] - Vol. 17, No. 1(2013)",1267-8694,"Vol. 10(2006) - Vol. 17, No. 1(2013)"
3327,"Vol. 15, supplement avril 2011",1267-8694,"Vol. 15, supplement avril 2011"
3328,"Vol. 16, supplement mars 2012",1267-8694,"Vol. 16, supplement mars 2012"


In [23]:
journals[journals.ISSN_NUMBER == '0937-9819']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
18059,21(2011)-25(2015),0937-9819,21(2011)-25(2015)
18060,10(2000)-15(2005),0937-9819,10(2000)-15(2005)
18061,17(2007)-18(2008) [lacunes],0937-9819,17(2007)-18(2008)
18062,1(1991)-7(1996/97),0937-9819,1(1991)-7(1996/97)


In [24]:
# convert multiple dates like 1996/97 : not used to prefer start year minimum
# pat_yearsmulti = (r'(?P<yone>[^0-9][12][0-9])[0-9][0-9]\/(?P<ytwo>[0-9][0-9])[^0-9]')
# journals['SUBFIELD_DATA'] = journals['SUBFIELD_DATA_BRUT'].str.replace(pat_yearsmulti, '', case=False)

In [25]:
# patterns for extract years
import re
# Start and end years included
pat_years1 = (r'(?P<syear>1[6789]\d\d|20[01]\d).*(?P<eyear>1[6789]\d\d|20[01]\d)')
# Only start year included and colletcion open
pat_years2 = (r'^.*(?P<yearopen>1[6789]\d\d|20[01]\d).*-(?!.*1[6789]\d\d|20[01]\d.*)$')
# Only one year included and colletcion not open
pat_years3 = (r'^[^-]+(?P<yearonly>1[6789]\d\d|20[01]\d)[^-]+$')
# Take first year in the string
pat_years4 = (r'(?P<firstyear>1[6789]\d\d|20[01]\d)')
# Only one year included and colletcion open with words after "-"
pat_years5 = (r'^.*(?P<yearopen2>1[6789]\d\d|20[01]\d).*-[ \>\.a-zA-Z\[\]\)]+$')

In [26]:
# Test some patterns
journals[journals.SUBFIELD_DATA.str.contains(pat_years5)]

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA
226,Vol 80(1998)- icpl,0035-8843,Vol 80(1998)- icpl
451,1985->,0290-439X,1985->
1591,4(1974)->,0074-770X,4(1974)->
3370,(1970)- ICPL,0012-3692,(1970)- ICPL
3834,Ed. 54(1990)->,0074-9613,Ed. 54(1990)->
4174,1(1981)->,0034-4427,1(1981)->
4212,Vol. 1(1929)->,0039-3223,Vol. 1(1929)->
4551,15(1946)->,0041-8994,15(1946)->
4907,Vol. 10(1941/43)->,0001-6969,Vol. 10(1941/43)->
4991,Vol. 1(1920)->,0016-2736,Vol. 1(1920)->


In [27]:
# Parsing string of colletion details
journals_years1 = journals['SUBFIELD_DATA'].str.extract(pat_years1, expand=False)
journals_years2 = journals['SUBFIELD_DATA'].str.extract(pat_years2, expand=False)
journals_years3 = journals['SUBFIELD_DATA'].str.extract(pat_years3, expand=False)
journals_years4 = journals['SUBFIELD_DATA'].str.extract(pat_years4, expand=False)
journals_years5 = journals['SUBFIELD_DATA'].str.extract(pat_years5, expand=False)

journals = pd.concat([journals, journals_years1], axis=1)
journals = pd.concat([journals, journals_years2], axis=1)
journals = pd.concat([journals, journals_years3], axis=1)
journals = pd.concat([journals, journals_years4], axis=1)
journals = pd.concat([journals, journals_years5], axis=1)


In [29]:
# Export data to check te extraction of years
journals.to_csv('data/temp/library/pjournals/pjournals_years.csv', sep='\t', encoding='utf-8', index=False)

In [30]:
journals[journals.ISSN_NUMBER == '0091-2174']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
10144,"Manquent: Vol. 45, no 2(2013)",0091-2174,,NaN,NaN,NaN,NaN,NaN,NaN
10145,Vol. 49 et 50,0091-2174,Vol. 49 et 50,NaN,NaN,NaN,NaN,NaN,NaN
10146,"Vol 51, no 1, 4 (2016)",0091-2174,"Vol 51, no 1, 4 (2016)",NaN,NaN,NaN,2016,2016,NaN
10147,Lacune: année 1974,0091-2174,,NaN,NaN,NaN,NaN,NaN,NaN
10148,"4(1973) - Vol. 51, no. 6 (2016)",0091-2174,"4(1973) - Vol. 51, no. 6 (2016)",1973,2016,NaN,NaN,1973,NaN


In [31]:
journals[journals.ISSN_NUMBER == '0937-9819']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
18059,21(2011)-25(2015),0937-9819,21(2011)-25(2015),2011,2015,NaN,NaN,2011,NaN
18060,10(2000)-15(2005),0937-9819,10(2000)-15(2005),2000,2005,NaN,NaN,2000,NaN
18061,17(2007)-18(2008) [lacunes],0937-9819,17(2007)-18(2008),2007,2008,NaN,NaN,2007,NaN
18062,1(1991)-7(1996/97),0937-9819,1(1991)-7(1996/97),1991,1996,NaN,NaN,1991,NaN


In [32]:
import numpy as np
# years = yearonly
journals['syear'] = np.where((journals['syear'].isnull()) & (journals['yearonly'].notnull()), journals['yearonly'], journals['syear'])
journals['eyear'] = np.where((journals['eyear'].isnull()) & (journals['yearonly'].notnull()), journals['yearonly'], journals['eyear'])

In [33]:
# eyear = 9999 if yearopen
journals['eyear'] = np.where(journals['yearopen'].notnull(), 9999, journals['eyear'])
journals['syear'] = np.where(journals['yearopen'].notnull(), journals['yearopen'], journals['syear'])

In [34]:
# test yearopen2
journals[journals.yearopen2.notnull()]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
226,Vol 80(1998)- icpl,0035-8843,Vol 80(1998)- icpl,NaN,NaN,NaN,NaN,1998,1998
451,1985->,0290-439X,1985->,NaN,NaN,NaN,NaN,1985,1985
1591,4(1974)->,0074-770X,4(1974)->,NaN,NaN,NaN,NaN,1974,1974
3370,(1970)- ICPL,0012-3692,(1970)- ICPL,NaN,NaN,NaN,NaN,1970,1970
3834,Ed. 54(1990)->,0074-9613,Ed. 54(1990)->,NaN,NaN,NaN,NaN,1990,1990
4174,1(1981)->,0034-4427,1(1981)->,NaN,NaN,NaN,NaN,1981,1981
4212,Vol. 1(1929)->,0039-3223,Vol. 1(1929)->,NaN,NaN,NaN,NaN,1929,1929
4551,15(1946)->,0041-8994,15(1946)->,NaN,NaN,NaN,NaN,1946,1946
4907,Vol. 10(1941/43)->,0001-6969,Vol. 10(1941/43)->,NaN,NaN,NaN,NaN,1941,1941
4991,Vol. 1(1920)->,0016-2736,Vol. 1(1920)->,NaN,NaN,NaN,NaN,1920,1920


In [35]:
# yearopen2 =>  9999 
journals['eyear'] = np.where(journals['yearopen2'].notnull(), 9999, journals['eyear'])
journals['syear'] = np.where(journals['yearopen2'].notnull(), journals['yearopen2'], journals['syear'])

In [36]:
# test first year if syear is null
journals[journals.firstyear.notnull() & journals.syear.isnull()]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
70,1969,0081-9638,1969,NaN,NaN,NaN,NaN,1969,NaN
71,1971,0081-9638,1971,NaN,NaN,NaN,NaN,1971,NaN
72,1973,0081-9638,1973,NaN,NaN,NaN,NaN,1973,NaN
73,1976,0081-9638,1976,NaN,NaN,NaN,NaN,1976,NaN
74,1980,0081-9638,1980,NaN,NaN,NaN,NaN,1980,NaN
276,2006/07,1661-111X,2006/07,NaN,NaN,NaN,NaN,2006,NaN
277,ed. 2004,0080-7400,ed. 2004,NaN,NaN,NaN,NaN,2004,NaN
278,no 247 2009,0151-0282,no 247 2009,NaN,NaN,NaN,NaN,2009,NaN
286,"55, No. 1-4, 6(2010)",0001-5512,"55, No. 1-4, 6(2010)",NaN,NaN,NaN,NaN,2010,NaN
305,"33,NO5-7,9-10(1947)",0002-9416,"33,NO5-7,9-10(1947)",NaN,NaN,NaN,NaN,1947,NaN


In [37]:
# years = firstyear
journals['syear'] = np.where((journals['syear'].isnull()) & (journals['firstyear'].notnull()), journals['firstyear'], journals['syear'])
journals['eyear'] = np.where((journals['eyear'].isnull()) & (journals['firstyear'].notnull()), journals['firstyear'], journals['eyear'])

In [38]:
journals[journals.ISSN_NUMBER == '0091-2174']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
10144,"Manquent: Vol. 45, no 2(2013)",0091-2174,,NaN,NaN,NaN,NaN,NaN,NaN
10145,Vol. 49 et 50,0091-2174,Vol. 49 et 50,NaN,NaN,NaN,NaN,NaN,NaN
10146,"Vol 51, no 1, 4 (2016)",0091-2174,"Vol 51, no 1, 4 (2016)",2016,2016,NaN,2016,2016,NaN
10147,Lacune: année 1974,0091-2174,,NaN,NaN,NaN,NaN,NaN,NaN
10148,"4(1973) - Vol. 51, no. 6 (2016)",0091-2174,"4(1973) - Vol. 51, no. 6 (2016)",1973,2016,NaN,NaN,1973,NaN


In [39]:
journals[journals.ISSN_NUMBER == '0937-9819']

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
18059,21(2011)-25(2015),0937-9819,21(2011)-25(2015),2011,2015,NaN,NaN,2011,NaN
18060,10(2000)-15(2005),0937-9819,10(2000)-15(2005),2000,2005,NaN,NaN,2000,NaN
18061,17(2007)-18(2008) [lacunes],0937-9819,17(2007)-18(2008),2007,2008,NaN,NaN,2007,NaN
18062,1(1991)-7(1996/97),0937-9819,1(1991)-7(1996/97),1991,1996,NaN,NaN,1991,NaN


In [40]:
# Export data to CSV to check treatments
journals.to_csv('data/temp/library/pjournals/pjournals_years_treatmentdone.csv', sep='\t', encoding='utf-8', index=False)

In [41]:
# Check journals without ISSN
journals[journals.ISSN_NUMBER.isnull()]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2


In [42]:
# Check journals without syear
journals[journals.syear.isnull()]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
38,incomplet,1499-3872,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
75,,0081-9638,,NaN,NaN,NaN,NaN,NaN,NaN
78,incomplet,0334-0236,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
81,incomplet,0748-7711,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
133,Incomplet,0309-3646,Incomplet,NaN,NaN,NaN,NaN,NaN,NaN
250,,0096-6908,,NaN,NaN,NaN,NaN,NaN,NaN
266,Incomplet,0555-4837,Incomplet,NaN,NaN,NaN,NaN,NaN,NaN
271,incomplet,1098-3511,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
273,Grand Dossier no 5,1777-375X,Grand Dossier no 5,NaN,NaN,NaN,NaN,NaN,NaN
275,Année courante,0397-4820,Année courante,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Check journals without eyear
journals[journals.eyear.isnull()]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2
38,incomplet,1499-3872,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
75,,0081-9638,,NaN,NaN,NaN,NaN,NaN,NaN
78,incomplet,0334-0236,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
81,incomplet,0748-7711,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
133,Incomplet,0309-3646,Incomplet,NaN,NaN,NaN,NaN,NaN,NaN
250,,0096-6908,,NaN,NaN,NaN,NaN,NaN,NaN
266,Incomplet,0555-4837,Incomplet,NaN,NaN,NaN,NaN,NaN,NaN
271,incomplet,1098-3511,incomplet,NaN,NaN,NaN,NaN,NaN,NaN
273,Grand Dossier no 5,1777-375X,Grand Dossier no 5,NaN,NaN,NaN,NaN,NaN,NaN
275,Année courante,0397-4820,Année courante,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Convert years to numeric type
journals['syearnum'] = pd.to_numeric(journals['syear'], errors='coerce')
journals['eyearnum'] = pd.to_numeric(journals['eyear'], errors='coerce')
journals

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2,syearnum,eyearnum
0,Vol. 55(1966) - 65(1976),0003-9772,Vol. 55(1966) - 65(1976),1966,1976,NaN,NaN,1966,NaN,1966.0,1976.0
1,Vol. 17(1985)- 40(2006),0041-1345,Vol. 17(1985)- 40(2006),1985,2006,NaN,NaN,1985,NaN,1985.0,2006.0
2,Vol. 16(1973)-40(1995),0071-7916,Vol. 16(1973)-40(1995),1973,1995,NaN,NaN,1973,NaN,1973.0,1995.0
3,Vol. 1(1961)-12(1973),0079-6824,Vol. 1(1961)-12(1973),1961,1973,NaN,NaN,1961,NaN,1961.0,1973.0
4,14(1974),0079-6824,14(1974),1974,1974,NaN,1974,1974,NaN,1974.0,1974.0
5,Vol. 8(1984)-19(1994) icpl,0266-7681,Vol. 8(1984)-19(1994) icpl,1984,1994,NaN,NaN,1984,NaN,1984.0,1994.0
6,20(1995)-37(2012),0266-7681,20(1995)-37(2012),1995,2012,NaN,NaN,1995,NaN,1995.0,2012.0
7,65-160 (1964-1976) icpl,0300-8827,65-160 (1964-1976) icpl,1964,1976,NaN,NaN,1964,NaN,1964.0,1976.0
8,167 (1977)-,0300-8827,167 (1977)-,1977,9999,1977,NaN,1977,NaN,1977.0,9999.0
9,Vol. 7(1982)-34(2009),0362-2436,Vol. 7(1982)-34(2009),1982,2009,NaN,NaN,1982,NaN,1982.0,2009.0


In [45]:
# Check journals with syear > eyear
journals[journals.syearnum > journals.eyearnum]

,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2,syearnum,eyearnum
308,31(1976)-30(1975),0003-6919,31(1976)-30(1975),1976,1975,NaN,NaN,1976,NaN,1976.0,1975.0
8770,Deel 36(1967)-37(1966/70),0075-8639,Deel 36(1967)-37(1966/70),1967,1966,NaN,NaN,1967,NaN,1967.0,1966.0
11320,23(1986)-24(1988) = n.s. 1(1959)-11(1967),0570-3077,23(1986)-24(1988) = n.s. 1(1959)-11(1967),1986,1967,NaN,NaN,1986,NaN,1986.0,1967.0
12278,anno 29(1998) - anno 31(1990),0516-6551,anno 29(1998) - anno 31(1990),1998,1990,NaN,NaN,1998,NaN,1998.0,1990.0
13008,4(1959)-12(1957),0079-3558,4(1959)-12(1957),1959,1957,NaN,NaN,1959,NaN,1959.0,1957.0
15234,"N⁰ 74(1987)-91(1989) = Vol. 11,1(1985)-12,5(1986)",0766-5725,"N⁰ 74(1987)-91(1989) = Vol. 11,1(1985)-12,5(1986)",1987,1986,NaN,NaN,1987,NaN,1987.0,1986.0
15572,"Vol. 3(1998/1989), cahiers 2-4",1633-9444,"Vol. 3(1998/1989), cahiers 2-4",1998,1989,NaN,NaN,1998,NaN,1998.0,1989.0
18080,1990/91-1922,0990-7939,1990/91-1922,1990,1922,NaN,NaN,1990,NaN,1990.0,1922.0
21017,Supplement 1960 (Selected Reprints from Volumes 1-4 (1953-1957)),0032-633X,Supplement 1960 (Selected Reprints from Volumes 1-4 (1953-1957)),1960,1957,NaN,NaN,1960,NaN,1960.0,1957.0
21591,Vol. 24(2001) relié avec vol.23(2000) [de l'ancien titre],0255-9005,Vol. 24(2001) relié avec vol.23(2000) [de l'ancien titre],2001,2000,NaN,2000,2001,NaN,2001.0,2000.0


In [46]:
# Fix problems to the rows with syear > eyear : inversion of years
journals.eyear.loc[journals.syearnum > journals.eyearnum] = journals.syearnum
journals.syear.loc[journals.syearnum > journals.eyearnum] = journals.eyearnum
journals[journals.syearnum > journals.eyearnum]

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,SUBFIELD_DATA_BRUT,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear,yearopen,yearonly,firstyear,yearopen2,syearnum,eyearnum
308,31(1976)-30(1975),0003-6919,31(1976)-30(1975),1975,1976,NaN,NaN,1976,NaN,1976.0,1975.0
8770,Deel 36(1967)-37(1966/70),0075-8639,Deel 36(1967)-37(1966/70),1966,1967,NaN,NaN,1967,NaN,1967.0,1966.0
11320,23(1986)-24(1988) = n.s. 1(1959)-11(1967),0570-3077,23(1986)-24(1988) = n.s. 1(1959)-11(1967),1967,1986,NaN,NaN,1986,NaN,1986.0,1967.0
12278,anno 29(1998) - anno 31(1990),0516-6551,anno 29(1998) - anno 31(1990),1990,1998,NaN,NaN,1998,NaN,1998.0,1990.0
13008,4(1959)-12(1957),0079-3558,4(1959)-12(1957),1957,1959,NaN,NaN,1959,NaN,1959.0,1957.0
15234,"N⁰ 74(1987)-91(1989) = Vol. 11,1(1985)-12,5(1986)",0766-5725,"N⁰ 74(1987)-91(1989) = Vol. 11,1(1985)-12,5(1986)",1986,1987,NaN,NaN,1987,NaN,1987.0,1986.0
15572,"Vol. 3(1998/1989), cahiers 2-4",1633-9444,"Vol. 3(1998/1989), cahiers 2-4",1989,1998,NaN,NaN,1998,NaN,1998.0,1989.0
18080,1990/91-1922,0990-7939,1990/91-1922,1922,1990,NaN,NaN,1990,NaN,1990.0,1922.0
21017,Supplement 1960 (Selected Reprints from Volumes 1-4 (1953-1957)),0032-633X,Supplement 1960 (Selected Reprints from Volumes 1-4 (1953-1957)),1957,1960,NaN,NaN,1960,NaN,1960.0,1957.0
21591,Vol. 24(2001) relié avec vol.23(2000) [de l'ancien titre],0255-9005,Vol. 24(2001) relié avec vol.23(2000) [de l'ancien titre],2000,2001,NaN,2000,2001,NaN,2001.0,2000.0


In [47]:
# Check lines not good (without years)
journals_bad = journals[journals.syear.isnull()][['ISSN_NUMBER', 'SUBFIELD_DATA', 'syear', 'eyear']]
journals_bad.to_csv('data/temp/library/pjournals/pjournals_bad.csv', sep='\t', encoding='utf-8', index=False)
journals_bad

,ISSN_NUMBER,SUBFIELD_DATA,syear,eyear
38,1499-3872,incomplet,NaN,NaN
75,0081-9638,,NaN,NaN
78,0334-0236,incomplet,NaN,NaN
81,0748-7711,incomplet,NaN,NaN
133,0309-3646,Incomplet,NaN,NaN
250,0096-6908,,NaN,NaN
266,0555-4837,Incomplet,NaN,NaN
271,1098-3511,incomplet,NaN,NaN
273,1777-375X,Grand Dossier no 5,NaN,NaN
275,0397-4820,Année courante,NaN,NaN


In [48]:
# Exclude rows without ISSN1
journals_issn = journals[journals.ISSN_NUMBER.notnull()][['ISSN_NUMBER', 'syear', 'eyear']]
# Exclude rows without years
journals_issn = journals_issn[journals_issn.syear.notnull()]

In [49]:
journals_issn = journals_issn[journals_issn.ISSN_NUMBER.notnull()]
journals_issn

,ISSN_NUMBER,syear,eyear
0,0003-9772,1966,1976
1,0041-1345,1985,2006
2,0071-7916,1973,1995
3,0079-6824,1961,1973
4,0079-6824,1974,1974
5,0266-7681,1984,1994
6,0266-7681,1995,2012
7,0300-8827,1964,1976
8,0300-8827,1977,9999
9,0362-2436,1982,2009


In [50]:
# enrich with ISSN-L (linking)
# The ISSN -> ISSN-L table has been obtained from www.issn.org (free to download but only after filling a request on their web site)
issns = pd.read_csv('data/sources/issnl/issn2issnl.csv', delimiter='\t', header=None, names=['issn', 'issnl'])
issns

,issn,issnl
0,0000-0019,0000-0019
1,0000-0027,0000-0027
2,0000-0043,0000-0043
3,0000-0051,0000-0051
4,0000-006X,0000-006X
5,0000-0078,0000-0078
6,0000-0094,0000-0094
7,0000-0108,0000-0108
8,0000-0140,0000-0140
9,0000-0159,0000-0159


In [51]:
# merge issns
# rename column to merge
journals_issn = journals_issn.rename(columns = {'ISSN_NUMBER': 'issn'})
journals_issn_merged = pd.merge(journals_issn, issns, on = 'issn', how='left')
journals_issn_merged[journals_issn_merged.issnl.notnull()]

,issn,syear,eyear,issnl
0,0003-9772,1966,1976,0003-9772
1,0041-1345,1985,2006,0041-1345
2,0071-7916,1973,1995,0071-7916
3,0079-6824,1961,1973,0079-6824
4,0079-6824,1974,1974,0079-6824
5,0266-7681,1984,1994,0266-7681
6,0266-7681,1995,2012,0266-7681
7,0300-8827,1964,1976,0300-8827
8,0300-8827,1977,9999,0300-8827
9,0362-2436,1982,2009,0362-2436


In [52]:
# check issns without issnl
journals_issn_merged_not_issnl = journals_issn_merged[journals_issn_merged.issnl.isnull()]
journals_issn_merged_not_issnl.to_csv('data/temp/library/pjournals/pjournals_not_issnl.csv', sep='\t', encoding='utf-8', index=False)
journals_issn_merged_not_issnl

,issn,syear,eyear,issnl
84,1547-4127,2006,2014,NaN
159,0077-0159,1964,1972,NaN
231,1044-1786,2002,2009,NaN
594,0091-1690,1972,1996,NaN
869,0567-8803,1967,1971,NaN
873,0921-884X,1953,1997,NaN
874,0921-884X,1951,1951,NaN
875,0921-884X,1949,1949,NaN
876,0921-884X,1982,1982,NaN
942,0015-556X,1956,1962,NaN


In [53]:
# Export ISSNs without ISSN-l deduped
journals_issn_merged_not_issnl_dedup = journals_issn_merged_not_issnl.sort_values('issn', ascending=False).drop_duplicates('issn').sort_index()
journals_issn_merged_not_issnl_dedup['issn'].to_csv('data/temp/library/pjournals/pjournals_not_issnl_dedup.csv', sep='\t', encoding='utf-8', index=False)
journals_issn_merged_not_issnl_dedup

,issn,syear,eyear,issnl
84,1547-4127,2006,2014,NaN
159,0077-0159,1964,1972,NaN
231,1044-1786,2002,2009,NaN
594,0091-1690,1972,1996,NaN
869,0567-8803,1967,1971,NaN
876,0921-884X,1982,1982,NaN
942,0015-556X,1956,1962,NaN
1043,0302-2773,1975,1988,NaN
1074,0372-7890,1902,1923,NaN
1080,0449-2846,1969,1976,NaN


In [54]:
# import ISSN-L file containing data checked manually for ISSN without ISSN-L (TODO)

In [55]:
# put ISSN at ISSN-L place if ISSN-L is empty
import numpy as np
journals_issn_merged.issnl.replace(np.NaN, journals_issn_merged.issn, inplace=True)
journals_issn_merged[journals_issn_merged.issnl.isnull()]

,issn,syear,eyear,issnl


In [56]:
# convert years to numeric format
journals_issn_merged['syearnum'] = pd.to_numeric(journals_issn_merged['syear'], errors='coerce')
journals_issn_merged['eyearnum'] = pd.to_numeric(journals_issn_merged['eyear'], errors='coerce')
journals_issn_merged

,issn,syear,eyear,issnl,syearnum,eyearnum
0,0003-9772,1966,1976,0003-9772,1966.0,1976.0
1,0041-1345,1985,2006,0041-1345,1985.0,2006.0
2,0071-7916,1973,1995,0071-7916,1973.0,1995.0
3,0079-6824,1961,1973,0079-6824,1961.0,1973.0
4,0079-6824,1974,1974,0079-6824,1974.0,1974.0
5,0266-7681,1984,1994,0266-7681,1984.0,1994.0
6,0266-7681,1995,2012,0266-7681,1995.0,2012.0
7,0300-8827,1964,1976,0300-8827,1964.0,1976.0
8,0300-8827,1977,9999,0300-8827,1977.0,9999.0
9,0362-2436,1982,2009,0362-2436,1982.0,2009.0


In [57]:
# check issns-l different than issn
journals_issn_merged[journals_issn_merged.issn != journals_issn_merged.issnl ]

,issn,syear,eyear,issnl,syearnum,eyearnum
37,1499-3872,2002,2005,2352-9377,2002.0,2005.0
89,0032-1052,1976,2008,0743-684X,1976.0,2008.0
124,1569-9293,2002,2010,1569-9285,2002.0,2010.0
141,0007-1226,1977,2005,0743-684X,1977.0,2005.0
152,1552-3365,1985,1985,0363-5465,1985.0,1985.0
153,1552-3365,1986,2012,0363-5465,1986.0,2012.0
699,1488-2159,1980,1980,0709-8936,1980.0,1980.0
700,1488-2159,1979,1979,0709-8936,1979.0,1979.0
701,1541-2016,1999,2005,1533-4058,1999.0,2005.0
932,0007-1226,1977,1998,0743-684X,1977.0,1998.0


In [58]:
# Keep num years only
pjournals = journals_issn_merged[['issn', 'issnl', 'syearnum', 'eyearnum']]
pjournals = pjournals.rename(columns = {'syearnum': 'syear', 'eyearnum': 'eyear'})
pjournals

,issn,issnl,syear,eyear
0,0003-9772,0003-9772,1966.0,1976.0
1,0041-1345,0041-1345,1985.0,2006.0
2,0071-7916,0071-7916,1973.0,1995.0
3,0079-6824,0079-6824,1961.0,1973.0
4,0079-6824,0079-6824,1974.0,1974.0
5,0266-7681,0266-7681,1984.0,1994.0
6,0266-7681,0266-7681,1995.0,2012.0
7,0300-8827,0300-8827,1964.0,1976.0
8,0300-8827,0300-8827,1977.0,9999.0
9,0362-2436,0362-2436,1982.0,2009.0


In [59]:
# check dates
pjournals[pjournals.syear > pjournals.eyear]

,issn,issnl,syear,eyear


In [60]:
# check dates
pjournals[pjournals.syear < 1700]

,issn,issnl,syear,eyear
4303,0021-7573,0021-7573,1678.0,1679.0
4304,0021-7573,0021-7573,1681.0,1681.0
4305,0021-7573,0021-7573,1683.0,1683.0
4306,0021-7573,0021-7573,1686.0,1686.0
4307,0021-7573,0021-7573,1691.0,1691.0
4308,0021-7573,0021-7573,1696.0,1696.0


In [61]:
# Export results to CSV
pjournals.to_csv('data/temp/library/pjournals/pjournals_ready_to_pubmed_merge.csv', sep='\t', encoding='utf-8', index=False)

In [62]:
# Export results with SUBFIELD_DATA_BRUT to compare and check the extraction
journals[['ISSN_NUMBER', 'SUBFIELD_DATA_BRUT', 'syear', 'eyear']].to_csv('data/temp/library/pjournals/pjournals_check.csv', sep='\t', encoding='utf-8', index=False)
journals[['ISSN_NUMBER', 'SUBFIELD_DATA_BRUT', 'syear', 'eyear']]

,ISSN_NUMBER,SUBFIELD_DATA_BRUT,syear,eyear
0,0003-9772,Vol. 55(1966) - 65(1976),1966,1976
1,0041-1345,Vol. 17(1985)- 40(2006),1985,2006
2,0071-7916,Vol. 16(1973)-40(1995),1973,1995
3,0079-6824,Vol. 1(1961)-12(1973),1961,1973
4,0079-6824,14(1974),1974,1974
5,0266-7681,Vol. 8(1984)-19(1994) icpl,1984,1994
6,0266-7681,20(1995)-37(2012),1995,2012
7,0300-8827,65-160 (1964-1976) icpl,1964,1976
8,0300-8827,167 (1977)-,1977,9999
9,0362-2436,Vol. 7(1982)-34(2009),1982,2009


## e-journals

The holdings of e-journals collection was exported by Pablo Iriarte from SFX on 23.10.2017 choosing:
* Output format: TXT
* Export which object types: Serials
* Export active portfolios with the following services: getFullTxt
* Export from: All targets
* Chinese Titles Export include the following (if exists): Simplified Chinese Title
* Include author information: unchecked
* Include note and authentication information: unchecked
* Include internal note information: unchecked
* Include alternative titles: unchecked

The name of fields are not supplied but are available here : https://knowledge.exlibrisgroup.com/SFX/Knowledge_Articles/Column_headings_for_SFX_advanced_export_-_TXT_type

0. sort_key
1. title
2. title_non_filing
2. issn
2. obj_id
2. target_public
2. threshold
2. eissn
2. abbrev_titles
2. target_service
2. lccn
2. portfolio_id
2. _856_u
2. _856_y
2. _852_a
2. _245_h
2. threshold_local
2. threshold_global
2. target_id
2. target_service_id
2. portfolio_id
2. cat_str
2. local
2. isbn
2. eisbn
2. publisher
2. place_of_pub
2. date_of_pub
2. object_type
2. def_avail
2. institute_id
2. institute_name
2. inst_avail
2. language
2. title_main
2. full_original_title
2. add_isbns
2. add_eisbns
2. authors_info
2. op_owner
2. thresholdLocal
2. parse_param
2. is_free
2. general_note
2. authentication_note
2. internal_note


In [1]:
# Extract title, ISSNs, free informations and year boundaries for licenced e-journals
# columns started at 0 so we have to subtract 1 to the numbers in the list above
import pandas as pd
pd.set_option('display.max_colwidth', -1)
sfx_orig = pd.read_csv('data/sources/library/ejournals/ecollection_20171023.csv', delimiter='\t', header=None, usecols=[1, 3, 5, 6, 7, 42], names=['title', 'issn', 'target', 'threshold', 'eissn', 'is_free'])
sfx_orig

,title,issn,target,threshold,eissn,is_free
0,Acta astronomica,0001-5237,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.,NaN,1.0
1,The Astronomical journal,0004-6256,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.,1538-3881,1.0
2,Publications of the Astronomical Society of Japan,0004-6264,Astrophysics Data System,Available from 1949 volume: 1.,2053-051X,1.0
3,Publications of the Astronomical Society of the Pacific,0004-6280,Astrophysics Data System,Available from 1889 volume: 1.,1538-3873,1.0
4,Astronomy and astrophysics,0004-6361,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.,1432-0746,1.0
5,Astrophysical journal,0004-637X,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1538-4357,1.0
6,Journal of the British Astronomical Association,0007-0297,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.,NaN,1.0
7,Observatory,0029-7704,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.,NaN,1.0
8,AAS photo-bulletin,0065-7433,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.,NaN,1.0
9,Proceedings - Astronomical Society of Australia,0066-9997,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.,0066-9977,1.0


In [2]:
# find duplicates
sfx_orig[sfx_orig.duplicated(keep=False)].sort_values(by=['title'])

,title,issn,target,threshold,eissn,is_free
108561,21世纪中学生作文(高中教师适用),1672-8327,TKN East View China Academic Journals Complete,Available in 2009.,NaN,0.0
109417,21世纪中学生作文(高中教师适用),1672-8327,TKN East View China Academic Journals Complete,Available in 2009.,NaN,0.0
48541,AFRICAN DEVELOPMENT REPORT,1607-8063,Free E- Journals,Available from 1998 until 2014.,NaN,1.0
42834,AFRICAN DEVELOPMENT REPORT,1607-8063,Free E- Journals,Available from 1998 until 2014.,NaN,1.0
96393,AHEAD OF THE CURVE,NaN,Lexis,Available from 1999 until 2001.,NaN,0.0
96394,AHEAD OF THE CURVE,NaN,Lexis,Available from 1999 until 2001.,NaN,0.0
92128,AHEAD OF THE CURVE,NaN,Lexis,Available from 1999 until 2001.,NaN,0.0
95885,AHEAD OF THE CURVE,NaN,Lexis,Available from 1999 until 2001.,NaN,0.0
94894,AIDS Alert,0887-0292,Lexis,Available from 1997.,0887-0772,0.0
94974,AIDS Alert,0887-0292,Lexis,Available from 1997.,0887-0772,0.0


In [3]:
# dedup
sfx_deduped = sfx_orig.drop_duplicates()
sfx_deduped

,title,issn,target,threshold,eissn,is_free
0,Acta astronomica,0001-5237,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.,NaN,1.0
1,The Astronomical journal,0004-6256,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.,1538-3881,1.0
2,Publications of the Astronomical Society of Japan,0004-6264,Astrophysics Data System,Available from 1949 volume: 1.,2053-051X,1.0
3,Publications of the Astronomical Society of the Pacific,0004-6280,Astrophysics Data System,Available from 1889 volume: 1.,1538-3873,1.0
4,Astronomy and astrophysics,0004-6361,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.,1432-0746,1.0
5,Astrophysical journal,0004-637X,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1538-4357,1.0
6,Journal of the British Astronomical Association,0007-0297,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.,NaN,1.0
7,Observatory,0029-7704,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.,NaN,1.0
8,AAS photo-bulletin,0065-7433,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.,NaN,1.0
9,Proceedings - Astronomical Society of Australia,0066-9997,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.,0066-9977,1.0


In [4]:
# Split lines with several availability status
# eg. 0004-640X (Available from 1968 volume: 1 until 1993 volume: 210. Available from 1995 volume: 223 until 1996 volume: 246.)  
# 1016-8478 (Available from 2000 volume: 10 issue: 1 until 2000 volume: 10 issue: 6. Available from 2009 volume: 27 issue: 1 until 2013 volume: 36 issue: 6.) 
sfx_deduped['threshold2'] = sfx_deduped.threshold.str.replace('\. Available', '£££Available', case=False)
sfx_deduped['threshold2'] = sfx_deduped.threshold2.str.split('£££')
sfx_deduped

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,title,issn,target,threshold,eissn,is_free,threshold2
0,Acta astronomica,0001-5237,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.,NaN,1.0,[Available from 1956 volume: 6 until 2006 volume: 56. ]
1,The Astronomical journal,0004-6256,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.,1538-3881,1.0,[Available from 1849 volume: 1 issue: 1. ]
2,Publications of the Astronomical Society of Japan,0004-6264,Astrophysics Data System,Available from 1949 volume: 1.,2053-051X,1.0,[Available from 1949 volume: 1. ]
3,Publications of the Astronomical Society of the Pacific,0004-6280,Astrophysics Data System,Available from 1889 volume: 1.,1538-3873,1.0,[Available from 1889 volume: 1. ]
4,Astronomy and astrophysics,0004-6361,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.,1432-0746,1.0,[Available from 1969 volume: 1 until 2000 volume: 364. ]
5,Astrophysical journal,0004-637X,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1538-4357,1.0,"[Available from 1895 volume: 1 issue: 1 until 1996 volume: 473, Available in 1999 volume: 525. ]"
6,Journal of the British Astronomical Association,0007-0297,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.,NaN,1.0,[Available from 1981 volume: 92 until 2002 volume: 112. ]
7,Observatory,0029-7704,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.,NaN,1.0,[Available from 1877 volume: 1 until 2013 volume: 133. ]
8,AAS photo-bulletin,0065-7433,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.,NaN,1.0,[Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1. ]
9,Proceedings - Astronomical Society of Australia,0066-9997,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.,0066-9977,1.0,[Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1. ]


In [5]:
sfx_deduped['threshold_len'] = sfx_deduped.threshold2.str.len()
sfx_deduped[sfx_deduped.threshold_len > 1].sort_values(by=['threshold_len'], ascending = False)

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,title,issn,target,threshold,eissn,is_free,threshold2,threshold_len
121819,Journal of pharmaceutical sciences,0022-3549,Elsevier Science Direct Open Access Journals,Available in 2007 volume: 96 issue: 5. Available in 2008 volume: 97 issue: 1. Available in 2009 volume: 98 issue: 9. Available in 2010 volume: 99 issue: 9. Available in 2011 volume: 100 issue: 1. Available in 2011 volume: 100 issue: 9. Available in 2012 volume: 101 issue: 9. Available in 2013 volume: 102 issue: 9. Available in 2014 volume: 103 issue: 9. Available in 2015 volume: 104 issue: 2. Available in 2015 volume: 104 issue: 9.,1520-6017,1.0,"[Available in 2007 volume: 96 issue: 5, Available in 2008 volume: 97 issue: 1, Available in 2009 volume: 98 issue: 9, Available in 2010 volume: 99 issue: 9, Available in 2011 volume: 100 issue: 1, Available in 2011 volume: 100 issue: 9, Available in 2012 volume: 101 issue: 9, Available in 2013 volume: 102 issue: 9, Available in 2014 volume: 103 issue: 9, Available in 2015 volume: 104 issue: 2, Available in 2015 volume: 104 issue: 9. ]",11.0
121731,Journal of adolescent health,1054-139X,Elsevier Science Direct Open Access Journals,Available in 2013 volume: 53 issue: 1. Available in 2013 volume: 53 issue: 4. Available in 2014 volume: 54 issue: 2. Available in 2014 volume: 54 issue: 3. Available in 2014 volume: 54 issue: 5. Available in 2015 volume: 56 issue: 1. Available in 2016 volume: 59 issue: 2. Available in 2016 volume: 59 issue: 4. Available in 2017 volume: 60 issue: 3. Available from 2017 volume: 61 issue: 4.,1879-1972,1.0,"[Available in 2013 volume: 53 issue: 1, Available in 2013 volume: 53 issue: 4, Available in 2014 volume: 54 issue: 2, Available in 2014 volume: 54 issue: 3, Available in 2014 volume: 54 issue: 5, Available in 2015 volume: 56 issue: 1, Available in 2016 volume: 59 issue: 2, Available in 2016 volume: 59 issue: 4, Available in 2017 volume: 60 issue: 3, Available from 2017 volume: 61 issue: 4. ]",10.0
121942,Phytomedicine,0944-7113,Elsevier Science Direct Open Access Journals,Available in 2006 volume: 13. Available from 2006 volume: 13 issue: 9 until 2006 volume: 13 issue: 10. Available in 2007 volume: 14. Available in 2007 volume: 14 issue: 10. Available in 2011 volume: 18. Available in 2011 volume: 18 issue: 13. Available in 2015 volume: 22. Available in 2015 volume: 22 issue: 12.,1618-095X,1.0,"[Available in 2006 volume: 13, Available from 2006 volume: 13 issue: 9 until 2006 volume: 13 issue: 10, Available in 2007 volume: 14, Available in 2007 volume: 14 issue: 10, Available in 2011 volume: 18, Available in 2011 volume: 18 issue: 13, Available in 2015 volume: 22, Available in 2015 volume: 22 issue: 12. ]",8.0
63840,情報知識学会誌,0917-1436,J-STAGE Free,Available in 1991 volume: 2 issue: 1. Available in 1994 volume: 4 issue: 1. Available in 1995 volume: 5 issue: 1. Available in 1997 volume: 7 issue: 1. Available from 1998 volume: 8 issue: 2 until 2000 volume: 10 issue: 1. Available from 2000 volume: 10 issue: 3 until 2001 volume: 11 issue: 4. Available from 2002 volume: 12 issue: 2 until 2003 volume: 13 issue: 2. Available from 2004 volume: 14 issue: 1 until 2017 volume: 27 issue: 2.,1881-7661,1.0,"[Available in 1991 volume: 2 issue: 1, Available in 1994 volume: 4 issue: 1, Available in 1995 volume: 5 issue: 1, Available in 1997 volume: 7 issue: 1, Available from 1998 volume: 8 issue: 2 until 2000 volume: 10 issue: 1, Available from 2000 volume: 10 issue: 3 until 2001 volume: 11 issue: 4, Available from 2002 volume: 12 issue: 2 until 2003 volume: 13 issue: 2, Available from 2004 volume: 14 issue: 1 until 2017 volume: 27 issue: 2. ]",8.0
116398,African geographical review,1937-6812,African Journal Archive Free,Available from 1910 volume: 1 issue: 1 until 1918 volume: 6 issue: 12. Available from 1942 issue: 73 until 1943 issue: 76. Available from 1946 issue: 87 until 1950 issue: 90. Available in 1958 issue: 99. Available in 1960. Available in 1962. Available from 1963 volume: 1 until 1974 volume: 12.,2163-264

In [6]:
# extract lines with multiple thresholds
sfx_multi = sfx_deduped[sfx_deduped.threshold_len > 1]
sfx_multi

,title,issn,target,threshold,eissn,is_free,threshold2,threshold_len
5,Astrophysical journal,0004-637X,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1538-4357,1.0,"[Available from 1895 volume: 1 issue: 1 until 1996 volume: 473, Available in 1999 volume: 525. ]",2.0
30,Astrophysics and Space Science,0004-640X,Astrophysics Data System,Available from 1968 volume: 1 until 1993 volume: 210. Available from 1995 volume: 223 until 1996 volume: 246.,1572-946X,1.0,"[Available from 1968 volume: 1 until 1993 volume: 210, Available from 1995 volume: 223 until 1996 volume: 246. ]",2.0
943,"Proceedings of the Royal Society A Mathematical, Physical and Engineering sciences",1364-5021,Highwire Press Free,Available from 1905 until 1944. Available from 2004. Most recent 24 month(s) not available.,1471-2946,1.0,"[Available from 1905 until 1944, Available from 2004. Most recent 24 month(s) not available. ]",2.0
948,"Philosophical transactions - Royal Society. Mathematical, Physical and engineering sciences",1364-503X,Highwire Press Free,Available from 1665 until 1943. Available from 2004 until 2013. Most recent 24 month(s) not available.,1471-2962,1.0,"[Available from 1665 until 1943, Available from 2004 until 2013. Most recent 24 month(s) not available. ]",2.0
1589,Zeitschrift für Analysis und ihre Anwendungen,0232-2064,Emis Free,Available from 1999 volume: 18 issue: 1 until 1999 volume: 18 issue: 4. Available in 2000 volume: 19 issue: 1.,1661-4534,1.0,"[Available from 1999 volume: 18 issue: 1 until 1999 volume: 18 issue: 4, Available in 2000 volume: 19 issue: 1. ]",2.0
1986,Molecules and Cells,1016-8478,Springer Standard Collection,Available from 2000 volume: 10 issue: 1 until 2000 volume: 10 issue: 6. Available from 2009 volume: 27 issue: 1 until 2013 volume: 36 issue: 6.,0219-1032,0.0,"[Available from 2000 volume: 10 issue: 1 until 2000 volume: 10 issue: 6, Available from 2009 volume: 27 issue: 1 until 2013 volume: 36 issue: 6. ]",2.0
3455,The HUGO Journal,1877-6558,Springer Standard Collection,Available from 2007 volume: 1 issue: 1. Available from 2015 volume: 9 issue: 1.,1877-6566,0.0,"[Available from 2007 volume: 1 issue: 1, Available from 2015 volume: 9 issue: 1. ]",2.0
27810,Organisation for Economic Cooperation and Development. The OECD Observer,0029-7054,Free E- Journals,Available in 1969. Available from 1995.,1561-5529,1.0,"[Available in 1969, Available from 1995. ]",2.0
27820,"Journal of information, law and technology",1361-4169,Free E- Journals,Available from 1996 issue: 1. Available in 2009 issue: 3.,NaN,1.0,"[Available from 1996 issue: 1, Available in 2009 issue: 3. ]",2.0
28209,Turtle and Tortoise Newsletter,1526-3096,Free E- Journals,Available from 2000 issue: 1 until 2002 issue: 6. Available in 2004 issue: 7.,1943-4189,1.0,"[Available from 2000 issue: 1 until 2002 issue: 6, Available in 2004 issue: 7. ]",2.0


In [7]:
# Explode rows based on threshold list
sfx_multi['index1'] = sfx_multi.index
sfx_multi = sfx_multi.set_index(['index1', 'title', 'issn', 'eissn', 'target', 'is_free', 'threshold'])['threshold2'].apply(pd.Series).stack()
sfx_multi = sfx_multi.reset_index()
sfx_multi.columns = ['index1', 'title', 'issn', 'eissn', 'target', 'is_free', 'threshold_orig', 'threshold_level', 'threshold']
sfx_multi

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,index1,title,issn,eissn,target,is_free,threshold_orig,threshold_level,threshold
0,5,Astrophysical journal,0004-637X,1538-4357,Astrophysics Data System,1.0,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,0,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473
1,5,Astrophysical journal,0004-637X,1538-4357,Astrophysics Data System,1.0,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1,Available in 1999 volume: 525.
2,30,Astrophysics and Space Science,0004-640X,1572-946X,Astrophysics Data System,1.0,Available from 1968 volume: 1 until 1993 volume: 210. Available from 1995 volume: 223 until 1996 volume: 246.,0,Available from 1968 volume: 1 until 1993 volume: 210
3,30,Astrophysics and Space Science,0004-640X,1572-946X,Astrophysics Data System,1.0,Available from 1968 volume: 1 until 1993 volume: 210. Available from 1995 volume: 223 until 1996 volume: 246.,1,Available from 1995 volume: 223 until 1996 volume: 246.
4,943,"Proceedings of the Royal Society A Mathematical, Physical and Engineering sciences",1364-5021,1471-2946,Highwire Press Free,1.0,Available from 1905 until 1944. Available from 2004. Most recent 24 month(s) not available.,0,Available from 1905 until 1944
5,943,"Proceedings of the Royal Society A Mathematical, Physical and Engineering sciences",1364-5021,1471-2946,Highwire Press Free,1.0,Available from 1905 until 1944. Available from 2004. Most recent 24 month(s) not available.,1,Available from 2004. Most recent 24 month(s) not available.
6,948,"Philosophical transactions - Royal Society. Mathematical, Physical and engineering sciences",1364-503X,1471-2962,Highwire Press Free,1.0,Available from 1665 until 1943. Available from 2004 until 2013. Most recent 24 month(s) not available.,0,Available from 1665 until 1943
7,948,"Philosophical transactions - Royal Society. Mathematical, Physical and engineering sciences",1364-503X,1471-2962,Highwire Press Free,1.0,Available from 1665 until 1943. Available from 2004 until 2013. Most recent 24 month(s) not available.,1,Available from 2004 until 2013. Most recent 24 month(s) not available.
8,1589,Zeitschrift für Analysis und ihre Anwendungen,0232-2064,1661-4534,Emis Free,1.0,Available from 1999 volume: 18 issue: 1 until 1999 volume: 18 issue: 4. Available in 2000 volume: 19 issue: 1.,0,Available from 1999 volume: 18 issue: 1 until 1999 volume: 18 issue: 4
9,1589,Zeitschrift für Analysis und ihre Anwendungen,0232-2064,1661-4534,Emis Free,1.0,Available from 1999 volume: 18 issue: 1 until 1999 volume: 18 issue: 4. Available in 2000 volume: 19 issue: 1.,1,Available in 2000 volume: 19 issue: 1.


In [8]:
sfx = sfx_deduped.append(sfx_multi)
sfx

,eissn,index1,is_free,issn,target,threshold,threshold2,threshold_len,threshold_level,threshold_orig,title
0,NaN,NaN,1.0,0001-5237,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.,[Available from 1956 volume: 6 until 2006 volume: 56. ],1.0,NaN,NaN,Acta astronomica
1,1538-3881,NaN,1.0,0004-6256,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.,[Available from 1849 volume: 1 issue: 1. ],1.0,NaN,NaN,The Astronomical journal
2,2053-051X,NaN,1.0,0004-6264,Astrophysics Data System,Available from 1949 volume: 1.,[Available from 1949 volume: 1. ],1.0,NaN,NaN,Publications of the Astronomical Society of Japan
3,1538-3873,NaN,1.0,0004-6280,Astrophysics Data System,Available from 1889 volume: 1.,[Available from 1889 volume: 1. ],1.0,NaN,NaN,Publications of the Astronomical Society of the Pacific
4,1432-0746,NaN,1.0,0004-6361,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.,[Available from 1969 volume: 1 until 2000 volume: 364. ],1.0,NaN,NaN,Astronomy and astrophysics
5,1538-4357,NaN,1.0,0004-637X,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,"[Available from 1895 volume: 1 issue: 1 until 1996 volume: 473, Available in 1999 volume: 525. ]",2.0,NaN,NaN,Astrophysical journal
6,NaN,NaN,1.0,0007-0297,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.,[Available from 1981 volume: 92 until 2002 volume: 112. ],1.0,NaN,NaN,Journal of the British Astronomical Association
7,NaN,NaN,1.0,0029-7704,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.,[Available from 1877 volume: 1 until 2013 volume: 133. ],1.0,NaN,NaN,Observatory
8,NaN,NaN,1.0,0065-7433,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.,[Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1. ],1.0,NaN,NaN,AAS photo-bulletin
9,0066-9977,NaN,1.0,0066-9997,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.,[Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1. ],1.0,NaN,NaN,Proceedings - Astronomical Society of Australia


In [9]:
# Embargoes and moving walls
sfx[sfx.threshold.notnull() & sfx.threshold.str.contains('recent')]

,eissn,index1,is_free,issn,target,threshold,threshold2,threshold_len,threshold_level,threshold_orig,title
412,1945-6182,NaN,0.0,1062-4783,JSTOR Arts and Sciences I,Available from 1992 volume: 26 issue: 1. Most recent 4 year(s) not available.,[Available from 1992 volume: 26 issue: 1. Most recent 4 year(s) not available. ],1.0,NaN,NaN,African American Review
414,2161-7953,NaN,0.0,0002-9300,JSTOR Arts and Sciences I,Available from 1907. Most recent 2 year(s) not available.,[Available from 1907. Most recent 2 year(s) not available. ],1.0,NaN,NaN,The American Journal of International Law
415,1080-6377,NaN,0.0,0002-9327,JSTOR Arts and Sciences I,Available from 1878 volume: 1 issue: 1. Most recent 6 year(s) not available.,[Available from 1878 volume: 1 issue: 1. Most recent 6 year(s) not available. ],1.0,NaN,NaN,American Journal of Mathematics
419,1930-0972,NaN,0.0,0002-9890,JSTOR Arts and Sciences I,Available from 1894 volume: 1 issue: 1. Most recent 4 year(s) not available.,[Available from 1894 volume: 1 issue: 1. Most recent 4 year(s) not available. ],1.0,NaN,NaN,The American Mathematical Monthly
420,1080-6490,NaN,0.0,0003-0678,JSTOR Arts and Sciences I,Available from 1949 volume: 1 issue: 1. Most recent 6 year(s) not available.,[Available from 1949 volume: 1 issue: 1. Most recent 6 year(s) not available. ],1.0,NaN,NaN,American Quarterly
423,2168-894X,NaN,0.0,0091-1798,JSTOR Arts and Sciences I,Available from 1973 volume: 1 issue: 1. Most recent 4 year(s) not available.,[Available from 1973 volume: 1 issue: 1. Most recent 4 year(s) not available. ],1.0,NaN,NaN,Annals of probability
424,2168-8966,NaN,0.0,0090-5364,JSTOR Arts and Sciences I,Available from 1973 volume: 1 issue: 1. Most recent 4 year(s) not available.,[Available from 1973 volume: 1 issue: 1. Most recent 4 year(s) not available. ],1.0,NaN,NaN,Annals of statistics
425,1545-4290,NaN,0.0,0084-6570,JSTOR Arts and Sciences I,Available from 1972 volume: 1. Most recent 6 year(s) not available.,[Available from 1972 volume: 1. Most recent 6 year(s) not available. ],1.0,NaN,NaN,Annual Review of Anthropology
428,1467-8322,NaN,0.0,0268-540X,JSTOR Arts and Sciences I,Available from 1985 volume: 1 issue: 1. Most recent 6 year(s) not available.,[Available from 1985 volume: 1 issue: 1. Most recent 6 year(s) not available. ],1.0,NaN,NaN,Anthropology today
433,1080-6512,NaN,0.0,0161-2492,JSTOR Arts and Sciences I,Available from 1976 issue: 1. Most recent 6 year(s) not available.,[Available from 1976 issue: 1. Most recent 6 year(s) not available. ],1.0,NaN,NaN,Callaloo


In [10]:
# Moving walls
sfx[sfx.threshold.notnull() & sfx.threshold.str.contains('year\(s\) available')]

,eissn,index1,is_free,issn,target,threshold,threshold2,threshold_len,threshold_level,threshold_orig,title
953,1471-2954,NaN,1.0,0962-8452,Highwire Press Free,Available from 1905 volume: 76 issue: 507 until 1947 volume: 135 issue: 838. Most recent 10 year(s) available. Most recent 1 year(s) not available.,[Available from 1905 volume: 76 issue: 507 until 1947 volume: 135 issue: 838. Most recent 10 year(s) available. Most recent 1 year(s) not available. ],1.0,NaN,NaN,Proceedings of the Royal Society B Biological Sciences
28012,NaN,NaN,1.0,0433-7050,Free E- Journals,Most recent 4 year(s) available.,[Most recent 4 year(s) available. ],1.0,NaN,NaN,Entscheidungen des Bundesarbeitsgerichts
28168,1943-0930,NaN,1.0,0036-8423,Free E- Journals,Most recent 1 year(s) available.,[Most recent 1 year(s) available. ],1.0,NaN,NaN,Science News
28814,NaN,NaN,1.0,1000-9140,Free E- Journals,Most recent 1 year(s) available.,[Most recent 1 year(s) available. ],1.0,NaN,NaN,Beijing Review
28979,NaN,NaN,1.0,1424-0904,Free E- Journals,Most recent 1 year(s) available.,[Most recent 1 year(s) available. ],1.0,NaN,NaN,Metrohm information
30205,1467-1093,NaN,1.0,1467-1085,Free E- Journals,Most recent 1 year(s) available.,[Most recent 1 year(s) available. ],1.0,NaN,NaN,Journal of Greco-Roman Christianity and Judaism
31851,NaN,NaN,1.0,1865-6072,Free E- Journals,Most recent 1 year(s) available.,[Most recent 1 year(s) available. ],1.0,NaN,NaN,Daimler-High-Tech-Report
33208,NaN,NaN,1.0,1369-7048,Free E- Journals,Most recent 5 year(s) available.,[Most recent 5 year(s) available. ],1.0,NaN,NaN,The alchemist
33994,NaN,NaN,1.0,0940-4163,Free E- Journals,Most recent 1 year(s) available.,[Most recent 1 year(s) available. ],1.0,NaN,NaN,Militärgeschichte
34207,1963-1006,NaN,1.0,0767-9513,Free E- Journals,Available from 1988 issue: 1. Most recent 5 year(s) available.,[Available from 1988 issue: 1. Most recent 5 year(s) available. ],1.0,NaN,NaN,"Hermès : cognition, communication, politique"


In [11]:
# Keep splited threshold only
ejournals = sfx[['title', 'issn', 'eissn', 'is_free', 'target', 'threshold']]
ejournals.reset_index(drop=True)
ejournals

,title,issn,eissn,is_free,target,threshold
0,Acta astronomica,0001-5237,NaN,1.0,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.
1,The Astronomical journal,0004-6256,1538-3881,1.0,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.
2,Publications of the Astronomical Society of Japan,0004-6264,2053-051X,1.0,Astrophysics Data System,Available from 1949 volume: 1.
3,Publications of the Astronomical Society of the Pacific,0004-6280,1538-3873,1.0,Astrophysics Data System,Available from 1889 volume: 1.
4,Astronomy and astrophysics,0004-6361,1432-0746,1.0,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.
5,Astrophysical journal,0004-637X,1538-4357,1.0,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.
6,Journal of the British Astronomical Association,0007-0297,NaN,1.0,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.
7,Observatory,0029-7704,NaN,1.0,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.
8,AAS photo-bulletin,0065-7433,NaN,1.0,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.
9,Proceedings - Astronomical Society of Australia,0066-9997,0066-9977,1.0,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.


In [12]:
# Extract years from string as Available from 1956 volume: 6 until 2006 volume: 56.
import re

# extract the first year
ejournals['syear'] = ejournals['threshold'].str.extract('Available from ([1-2][0-9][0-9][0-9])', expand=True)

# extract the year in case of one year only
ejournals['oneyear'] = ejournals['threshold'].str.extract('Available in ([1-2][0-9][0-9][0-9])', expand=True)

# extract the last year
ejournals['eyear'] = ejournals['threshold'].str.extract('until ([1-2][0-9][0-9][0-9])', expand=True)

ejournals

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear
0,Acta astronomica,0001-5237,NaN,1.0,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.,1956,NaN,2006
1,The Astronomical journal,0004-6256,1538-3881,1.0,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.,1849,NaN,NaN
2,Publications of the Astronomical Society of Japan,0004-6264,2053-051X,1.0,Astrophysics Data System,Available from 1949 volume: 1.,1949,NaN,NaN
3,Publications of the Astronomical Society of the Pacific,0004-6280,1538-3873,1.0,Astrophysics Data System,Available from 1889 volume: 1.,1889,NaN,NaN
4,Astronomy and astrophysics,0004-6361,1432-0746,1.0,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.,1969,NaN,2000
5,Astrophysical journal,0004-637X,1538-4357,1.0,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1895,1999,1996
6,Journal of the British Astronomical Association,0007-0297,NaN,1.0,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.,1981,NaN,2002
7,Observatory,0029-7704,NaN,1.0,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.,1877,NaN,2013
8,AAS photo-bulletin,0065-7433,NaN,1.0,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.,1969,NaN,1986
9,Proceedings - Astronomical Society of Australia,0066-9997,0066-9977,1.0,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.,1976,NaN,1994


In [16]:
import numpy as np
# replace start year with 999 for collections only with end year
ejournals.loc[ejournals.syear.isnull() & ejournals.eyear.notnull(),'syear'] = 999

# replace end year for periods open
ejournals.loc[ejournals.syear.notnull() & ejournals.eyear.isnull(),'eyear'] = 9999

# replace start year and end years with oneyear if they are empty
ejournals['syear'] = np.where((ejournals['syear'].isnull()) & (ejournals['oneyear'].notnull()), ejournals['oneyear'], ejournals['syear'])
ejournals['eyear'] = np.where((ejournals['eyear'].isnull()) & (ejournals['oneyear'].notnull()), ejournals['oneyear'], ejournals['eyear'])

# check the dataframe
ejournals

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear
0,Acta astronomica,0001-5237,NaN,1.0,Astrophysics Data System,Available from 1956 volume: 6 until 2006 volume: 56.,1956,NaN,2006
1,The Astronomical journal,0004-6256,1538-3881,1.0,Astrophysics Data System,Available from 1849 volume: 1 issue: 1.,1849,NaN,9999
2,Publications of the Astronomical Society of Japan,0004-6264,2053-051X,1.0,Astrophysics Data System,Available from 1949 volume: 1.,1949,NaN,9999
3,Publications of the Astronomical Society of the Pacific,0004-6280,1538-3873,1.0,Astrophysics Data System,Available from 1889 volume: 1.,1889,NaN,9999
4,Astronomy and astrophysics,0004-6361,1432-0746,1.0,Astrophysics Data System,Available from 1969 volume: 1 until 2000 volume: 364.,1969,NaN,2000
5,Astrophysical journal,0004-637X,1538-4357,1.0,Astrophysics Data System,Available from 1895 volume: 1 issue: 1 until 1996 volume: 473. Available in 1999 volume: 525.,1895,1999,1996
6,Journal of the British Astronomical Association,0007-0297,NaN,1.0,Astrophysics Data System,Available from 1981 volume: 92 until 2002 volume: 112.,1981,NaN,2002
7,Observatory,0029-7704,NaN,1.0,Astrophysics Data System,Available from 1877 volume: 1 until 2013 volume: 133.,1877,NaN,2013
8,AAS photo-bulletin,0065-7433,NaN,1.0,Astrophysics Data System,Available from 1969 volume: 1 issue: 1 until 1986 volume: 43 issue: 1.,1969,NaN,1986
9,Proceedings - Astronomical Society of Australia,0066-9997,0066-9977,1.0,Astrophysics Data System,Available from 1976 volume: 3 issue: 1 until 1994 volume: 11 issue: 1.,1976,NaN,1994


In [17]:
# check lines without threshold
ejournals.loc[ejournals.threshold.isnull()]

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear
774,Current protocols in molecular biology,1934-3639,1934-3647,0.0,Wiley Online Library,NaN,NaN,NaN,NaN
805,Spatium,1450-569X,2217-8066,0.0,De Gruyter Online Journals,NaN,NaN,NaN,NaN
839,Romanische Bibliographie Online Datenbank,NaN,2193-3030,0.0,De Gruyter Online Journals,NaN,NaN,NaN,NaN
1486,Physical review,0031-899X,1536-6065,0.0,American Physical Society Journals,NaN,NaN,NaN,NaN
1541,Journal of physical chemistry,0092-7325,1943-300X,0.0,American Chemical Society Journals,NaN,NaN,NaN,NaN
1625,Mathematical Collections and Conference Proceedings,NaN,NaN,1.0,Emis Free,NaN,NaN,NaN,NaN
1626,"Classical Works, Selecta, and Opera Omnia",NaN,NaN,1.0,Emis Free,NaN,NaN,NaN,NaN
3464,Cell Stress & Chaperones,1355-8145,1466-1268,0.0,Springer Standard Collection,NaN,NaN,NaN,NaN
4299,The Internet Journal of Medical Technology,1559-4610,NaN,1.0,Internet Scientific Publications,NaN,NaN,NaN,NaN
4300,The Internet journal of parasitic diseases,1559-4629,NaN,1.0,Internet Scientific Publications,NaN,NaN,NaN,NaN


In [18]:
# replace start year with 999 for collections without threshold
ejournals.loc[ejournals.threshold.isnull(),'syear'] = 999

# replace end year for periods open
ejournals.loc[ejournals.threshold.isnull(),'eyear'] = 9999

ejournals.loc[ejournals.threshold.isnull()]

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear
774,Current protocols in molecular biology,1934-3639,1934-3647,0.0,Wiley Online Library,NaN,999,NaN,9999
805,Spatium,1450-569X,2217-8066,0.0,De Gruyter Online Journals,NaN,999,NaN,9999
839,Romanische Bibliographie Online Datenbank,NaN,2193-3030,0.0,De Gruyter Online Journals,NaN,999,NaN,9999
1486,Physical review,0031-899X,1536-6065,0.0,American Physical Society Journals,NaN,999,NaN,9999
1541,Journal of physical chemistry,0092-7325,1943-300X,0.0,American Chemical Society Journals,NaN,999,NaN,9999
1625,Mathematical Collections and Conference Proceedings,NaN,NaN,1.0,Emis Free,NaN,999,NaN,9999
1626,"Classical Works, Selecta, and Opera Omnia",NaN,NaN,1.0,Emis Free,NaN,999,NaN,9999
3464,Cell Stress & Chaperones,1355-8145,1466-1268,0.0,Springer Standard Collection,NaN,999,NaN,9999
4299,The Internet Journal of Medical Technology,1559-4610,NaN,1.0,Internet Scientific Publications,NaN,999,NaN,9999
4300,The Internet journal of parasitic diseases,1559-4629,NaN,1.0,Internet Scientific Publications,NaN,999,NaN,9999


In [19]:
# check missing dates
ejournals.loc[ejournals.eyear.isnull()]

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear
5821,Nordic journal of international law,0902-7351,1571-8107,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN
5827,International negotiation,1382-340X,1571-8069,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN
12353,Country Commerce. Russia,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN
12354,Country Report: Palestinian Territories,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN
12355,Country Report: Vanuatu,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN
12356,Food & Drinks Forecast Eastern Europe,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN
28012,Entscheidungen des Bundesarbeitsgerichts,0433-7050,NaN,1.0,Free E- Journals,Most recent 4 year(s) available.,NaN,NaN,NaN
28168,Science News,0036-8423,1943-0930,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN
28814,Beijing Review,1000-9140,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN
28979,Metrohm information,1424-0904,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN


In [20]:
# Embargoes : Most recent X year(s) not available. OR Most recent X month(s) not available. OR  Most recent 1 year(s) 6 month(s) not available.
ejournals['embargoy'] = ejournals['threshold'].str.extract('Most recent ([0-9]+) year\(s\) not available', expand=True)
ejournals['embargom'] = ejournals['threshold'].str.extract('Most recent ([0-9]+) month\(s\) not available', expand=True)
ejournals['embargoymy'] = ejournals['threshold'].str.extract('Most recent ([0-9]+) year\(s\) [0-9]+ month\(s\) not available', expand=True)
ejournals['embargoymm'] = ejournals['threshold'].str.extract('Most recent [0-9]+ year\(s\) ([0-9]+) month\(s\) not available', expand=True)
ejournals['embargoy'] = np.where((ejournals['embargoymy'].notnull()), ejournals['embargoymy'], ejournals['embargoy'])
ejournals['embargom'] = np.where((ejournals['embargoymm'].notnull()), ejournals['embargoymm'], ejournals['embargom'])

# Moving walls : Most recent X year(s) available OR Most recent X month(s)
ejournals['mwally'] = ejournals['threshold'].str.extract('Most recent ([0-9]+) year\(s\) available', expand=True)
ejournals['mwallm'] = ejournals['threshold'].str.extract('Most recent ([0-9]+) month\(s\) available', expand=True)

ejournals[ejournals.syear.isnull() & sfx.threshold.notnull()]

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear,embargoy,embargom,embargoymy,embargoymm,mwally,mwallm
5821,Nordic journal of international law,0902-7351,1571-8107,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
5827,International negotiation,1382-340X,1571-8069,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
12353,Country Commerce. Russia,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
12354,Country Report: Palestinian Territories,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
12355,Country Report: Vanuatu,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
12356,Food & Drinks Forecast Eastern Europe,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
28012,Entscheidungen des Bundesarbeitsgerichts,0433-7050,NaN,1.0,Free E- Journals,Most recent 4 year(s) available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
28168,Science News,0036-8423,1943-0930,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
28814,Beijing Review,1000-9140,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
28979,Metrohm information,1424-0904,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [21]:
# convert string to ints
ejournals['syearnum'] = pd.to_numeric(ejournals['syear'], errors='coerce')
ejournals['eyearnum'] = pd.to_numeric(ejournals['eyear'], errors='coerce')
ejournals['embargoynum'] = pd.to_numeric(ejournals['embargoy'], errors='coerce')
ejournals['embargomnum'] = pd.to_numeric(ejournals['embargom'], errors='coerce')
ejournals['embargoymmnum'] = pd.to_numeric(ejournals['embargoymm'], errors='coerce')
ejournals['embargoymynum'] = pd.to_numeric(ejournals['embargoymy'], errors='coerce')
ejournals['mwallynum'] = pd.to_numeric(ejournals['mwally'], errors='coerce')
ejournals['mwallmnum'] = pd.to_numeric(ejournals['mwallm'], errors='coerce')

# Adjust years if embargo
ejournals['eyearnum'] = np.where((ejournals['embargoy'].notnull()), 2017 - ejournals['embargoynum'], ejournals['eyearnum'])
ejournals['syearnum'] = np.where((ejournals['embargoy'].notnull() & ejournals['syear'].isnull()), 999, ejournals['syearnum'])
ejournals['eyearnum'] = np.where((ejournals['embargom'].notnull()), 2017 - (ejournals['embargomnum'] / 12), ejournals['eyearnum'])
ejournals['syearnum'] = np.where((ejournals['embargom'].notnull() & ejournals['syear'].isnull()), 999, ejournals['syearnum'])

# Adjust years if moving wall
ejournals['syearnum'] = np.where((ejournals['mwally'].notnull() & ejournals['syear'].isnull()), 2017 - ejournals['mwallynum'], ejournals['syearnum'])
ejournals['eyearnum'] = np.where((ejournals['mwally'].notnull() & ejournals['eyear'].isnull()), 9999, ejournals['eyearnum'])
ejournals['syearnum'] = np.where((ejournals['mwallm'].notnull() & ejournals['syear'].isnull()), 2017 - (ejournals['mwallmnum'] / 12), ejournals['syearnum'])
ejournals['eyearnum'] = np.where((ejournals['mwallm'].notnull() & ejournals['eyear'].isnull()), 9999, ejournals['eyearnum'])

# If no years and "Available from issue: 1"
ejournals['syearnum'] = np.where((ejournals['syear'].isnull() & ejournals['threshold'].str.contains('Available from issue: 1')), 999, ejournals['syearnum'])
ejournals['eyearnum'] = np.where((ejournals['eyear'].isnull() & ejournals['threshold'].str.contains('Available from issue: 1')), 9999, ejournals['eyearnum'])

ejournals[ejournals.syear.isnull() & ejournals.threshold.notnull()]

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear,embargoy,...,mwally,mwallm,syearnum,eyearnum,embargoynum,embargomnum,embargoymmnum,embargoymynum,mwallynum,mwallmnum
5821,Nordic journal of international law,0902-7351,1571-8107,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN,1,...,NaN,NaN,999.000000,2016.000000,1.0,NaN,NaN,NaN,NaN,NaN
5827,International negotiation,1382-340X,1571-8069,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN,1,...,NaN,NaN,999.000000,2016.000000,1.0,NaN,NaN,NaN,NaN,NaN
12353,Country Commerce. Russia,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
12354,Country Report: Palestinian Territories,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
12355,Country Report: Vanuatu,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
12356,Food & Drinks Forecast Eastern Europe,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
28012,Entscheidungen des Bundesarbeitsgerichts,0433-7050,NaN,1.0,Free E- Journals,Most recent 4 year(s) available.,NaN,NaN,NaN,NaN,...,4,NaN,2013.000000,9999.000000,NaN,NaN,NaN,NaN,4.0,NaN
28168,Science News,0036-8423,1943-0930,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,...,1,NaN,2016.000000,9999.000000,NaN,NaN,NaN,NaN,1.0,NaN
28814,Beijing Review,1000-9140,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,...,1,NaN,2016.000000,9999.000000,NaN,NaN,NaN,NaN,1.0,NaN
28979,Metrohm information,1424-0904,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,...,1,NaN,2016.000000,9999.000000,NaN,NaN,NaN,NaN,1.0,NaN


In [22]:
# Ex with multiple parts : 1016-8478 Available from 2000 volume: 10 issue: 1 until 2000 volume: 10 issue: 6. Available from 2009 volume: 27 issue: 1 until 2013 volume: 36 issue: 6.
ejournals[ejournals.issn == '1016-8478']

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear,embargoy,...,mwally,mwallm,syearnum,eyearnum,embargoynum,embargomnum,embargoymmnum,embargoymynum,mwallynum,mwallmnum
1986,Molecules and Cells,1016-8478,0219-1032,0.0,Springer Standard Collection,Available from 2000 volume: 10 issue: 1 until 2000 volume: 10 issue: 6. Available from 2009 volume: 27 issue: 1 until 2013 volume: 36 issue: 6.,2000,NaN,2000,NaN,...,NaN,NaN,2000.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN
27978,Molecules and Cells,1016-8478,0219-1032,1.0,Free E- Journals,Available from 1990 volume: 1 issue: 1.,1990,NaN,9999,NaN,...,NaN,NaN,1990.0,9999.0,NaN,NaN,NaN,NaN,NaN,NaN
66980,Molecules and Cells,1016-8478,0219-1032,1.0,PubMed Central,Available from 2011 volume: 31.,2011,NaN,9999,NaN,...,NaN,NaN,2011.0,9999.0,NaN,NaN,NaN,NaN,NaN,NaN
10,Molecules and Cells,1016-8478,0219-1032,0.0,Springer Standard Collection,Available from 2000 volume: 10 issue: 1 until 2000 volume: 10 issue: 6,2000,NaN,2000,NaN,...,NaN,NaN,2000.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN
11,Molecules and Cells,1016-8478,0219-1032,0.0,Springer Standard Collection,Available from 2009 volume: 27 issue: 1 until 2013 volume: 36 issue: 6.,2009,NaN,2013,NaN,...,NaN,NaN,2009.0,2013.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Export to check
ejournals.to_csv('data/temp/library/ejournals/ejournals_check_collection_extract.csv', sep='\t', encoding='utf-8', index=False)

In [24]:
# Check syear empty
ejournals[ejournals.syear.isnull() & ejournals.threshold.notnull()]

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear,embargoy,...,mwally,mwallm,syearnum,eyearnum,embargoynum,embargomnum,embargoymmnum,embargoymynum,mwallynum,mwallmnum
5821,Nordic journal of international law,0902-7351,1571-8107,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN,1,...,NaN,NaN,999.000000,2016.000000,1.0,NaN,NaN,NaN,NaN,NaN
5827,International negotiation,1382-340X,1571-8069,0.0,EBSCOhost Business Source Premier,Most recent 1 year(s) not available.,NaN,NaN,NaN,1,...,NaN,NaN,999.000000,2016.000000,1.0,NaN,NaN,NaN,NaN,NaN
12353,Country Commerce. Russia,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
12354,Country Report: Palestinian Territories,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
12355,Country Report: Vanuatu,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
12356,Food & Drinks Forecast Eastern Europe,NaN,NaN,0.0,EBSCOhost Business Source Premier,Most recent 6 month(s) not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,999.000000,2016.500000,NaN,6.0,NaN,NaN,NaN,NaN
28012,Entscheidungen des Bundesarbeitsgerichts,0433-7050,NaN,1.0,Free E- Journals,Most recent 4 year(s) available.,NaN,NaN,NaN,NaN,...,4,NaN,2013.000000,9999.000000,NaN,NaN,NaN,NaN,4.0,NaN
28168,Science News,0036-8423,1943-0930,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,...,1,NaN,2016.000000,9999.000000,NaN,NaN,NaN,NaN,1.0,NaN
28814,Beijing Review,1000-9140,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,...,1,NaN,2016.000000,9999.000000,NaN,NaN,NaN,NaN,1.0,NaN
28979,Metrohm information,1424-0904,NaN,1.0,Free E- Journals,Most recent 1 year(s) available.,NaN,NaN,NaN,NaN,...,1,NaN,2016.000000,9999.000000,NaN,NaN,NaN,NaN,1.0,NaN


In [25]:
# test embargo in 0902-7351
ejournals[ejournals.issn == '0902-7351'][['issn', 'threshold', 'syearnum', 'eyearnum']]

,issn,threshold,syearnum,eyearnum
5821,0902-7351,Most recent 1 year(s) not available.,999.0,2016.0
59587,0902-7351,Available from 1930 volume: 1. Most recent 2 year(s) not available.,1930.0,2015.0
111927,0902-7351,Available from 1947 volume: 18 issue: 1.,1947.0,9999.0


In [26]:
# Check syearnum empty
ejournals[ejournals.syearnum.isnull() & ejournals.threshold.notnull()]

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear,embargoy,...,mwally,mwallm,syearnum,eyearnum,embargoynum,embargomnum,embargoymmnum,embargoymynum,mwallynum,mwallmnum
89270,Research Series / International Institute for Labour Studies,NaN,NaN,0.0,Ressources électroniques de l'IHEID,Available from issue: 89.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89295,"Economic Survey - Economic Division, Ministry of Finance, India",0536-9363,NaN,0.0,Ressources électroniques de l'IHEID,Most recent not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Check eyearnum empty
ejournals[ejournals.eyearnum.isnull() & ejournals.threshold.notnull()]

,title,issn,eissn,is_free,target,threshold,syear,oneyear,eyear,embargoy,...,mwally,mwallm,syearnum,eyearnum,embargoynum,embargomnum,embargoymmnum,embargoymynum,mwallynum,mwallmnum
89270,Research Series / International Institute for Labour Studies,NaN,NaN,0.0,Ressources électroniques de l'IHEID,Available from issue: 89.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89295,"Economic Survey - Economic Division, Ministry of Finance, India",0536-9363,NaN,0.0,Ressources électroniques de l'IHEID,Most recent not available.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Exclude rows without ISSN1
ejournals_issn1 = ejournals[ejournals.issn.notnull()][['issn', 'syearnum', 'eyearnum', 'is_free']]

# Exclude rows without years
ejournals_issn1 = ejournals_issn1[ejournals_issn1.syearnum.notnull()]

# convert years to int
ejournals_issn1['syear'] = ejournals_issn1['syearnum'].astype(int)
ejournals_issn1['eyear'] = ejournals_issn1['eyearnum'].astype(int)

ejournals_issn1 = ejournals_issn1[['issn', 'syear', 'eyear', 'is_free']]

ejournals_issn1

,issn,syear,eyear,is_free
0,0001-5237,1956,2006,1.0
1,0004-6256,1849,9999,1.0
2,0004-6264,1949,9999,1.0
3,0004-6280,1889,9999,1.0
4,0004-6361,1969,2000,1.0
5,0004-637X,1895,1996,1.0
6,0007-0297,1981,2002,1.0
7,0029-7704,1877,2013,1.0
8,0065-7433,1969,1986,1.0
9,0066-9997,1976,1994,1.0


In [29]:
# take also ISSN2
ejournals_issn2 = ejournals[ejournals.eissn.notnull() & (ejournals.eissn != ejournals.issn)][['eissn', 'syearnum', 'eyearnum', 'is_free']]

# rename column eissn
ejournals_issn2.rename(columns={'eissn': 'issn'}, inplace=True)

# Exclude rows without years
ejournals_issn2 = ejournals_issn2[ejournals_issn2.syearnum.notnull()]

# convert years to int
ejournals_issn2['syear'] = ejournals_issn2['syearnum'].astype(int)
ejournals_issn2['eyear'] = ejournals_issn2['eyearnum'].astype(int)

ejournals_issn2 = ejournals_issn2[ejournals_issn2.issn.notnull()][['issn', 'syear', 'eyear', 'is_free']]
ejournals_issn2

,issn,syear,eyear,is_free
1,1538-3881,1849,9999,1.0
2,2053-051X,1949,9999,1.0
3,1538-3873,1889,9999,1.0
4,1432-0746,1969,2000,1.0
5,1538-4357,1895,1996,1.0
9,0066-9977,1976,1994,1.0
10,1538-4365,1970,9999,1.0
13,2249-9601,1973,2014,1.0
14,1286-4846,1970,2000,1.0
17,1562-6873,1975,2003,1.0


In [30]:
ejournals_issn_all = ejournals_issn1.append(ejournals_issn2, ignore_index=True)
ejournals_issn_all

,issn,syear,eyear,is_free
0,0001-5237,1956,2006,1.0
1,0004-6256,1849,9999,1.0
2,0004-6264,1949,9999,1.0
3,0004-6280,1889,9999,1.0
4,0004-6361,1969,2000,1.0
5,0004-637X,1895,1996,1.0
6,0007-0297,1981,2002,1.0
7,0029-7704,1877,2013,1.0
8,0065-7433,1969,1986,1.0
9,0066-9997,1976,1994,1.0


In [31]:
ejournals_issn_all = ejournals_issn_all[ejournals_issn_all.issn.notnull()]

In [32]:
ejournals_issn_all

,issn,syear,eyear,is_free
0,0001-5237,1956,2006,1.0
1,0004-6256,1849,9999,1.0
2,0004-6264,1949,9999,1.0
3,0004-6280,1889,9999,1.0
4,0004-6361,1969,2000,1.0
5,0004-637X,1895,1996,1.0
6,0007-0297,1981,2002,1.0
7,0029-7704,1877,2013,1.0
8,0065-7433,1969,1986,1.0
9,0066-9997,1976,1994,1.0


In [34]:
# enrich with ISSN-L (linking)
# The ISSN -> ISSN-L table has been obtained from www.issn.org (free to download but only after filling a request on their web site)
issns = pd.read_csv('data/sources/issnl/issn2issnl.csv', delimiter='\t', header=None, names=['issn', 'issnl'])
issns

,issn,issnl
0,0000-0019,0000-0019
1,0000-0027,0000-0027
2,0000-0043,0000-0043
3,0000-0051,0000-0051
4,0000-006X,0000-006X
5,0000-0078,0000-0078
6,0000-0094,0000-0094
7,0000-0108,0000-0108
8,0000-0140,0000-0140
9,0000-0159,0000-0159


In [35]:
# Enrich with ISSN-L
ejournals_issn_all_merged = pd.merge(ejournals_issn_all, issns, on = 'issn', how='left')
ejournals_issn_all_merged[ejournals_issn_all_merged.issnl.notnull()]

,issn,syear,eyear,is_free,issnl
0,0001-5237,1956,2006,1.0,0001-5237
1,0004-6256,1849,9999,1.0,0004-6256
2,0004-6264,1949,9999,1.0,0004-6264
3,0004-6280,1889,9999,1.0,0004-6280
4,0004-6361,1969,2000,1.0,0004-6361
5,0004-637X,1895,1996,1.0,0004-637X
6,0007-0297,1981,2002,1.0,0007-0297
7,0029-7704,1877,2013,1.0,0029-7704
8,0065-7433,1969,1986,1.0,0065-7433
9,0066-9997,1976,1994,1.0,0066-9997


In [36]:
# check issns without issnl
ejournals_issn_all_merged_not_issnl = ejournals_issn_all_merged[ejournals_issn_all_merged.issnl.isnull()]
ejournals_issn_all_merged_not_issnl.to_csv('data/temp/library/ejournals/ejournals_not_issnl.csv', sep='\t', encoding='utf-8', index=False)
ejournals_issn_all_merged_not_issnl

,issn,syear,eyear,is_free,issnl
16,0862-920X,1991,1998,1.0,NaN
214,2296-7389,2014,9999,0.0,NaN
254,1748-9318,2006,9999,0.0,NaN
255,1749-4680,2008,9999,0.0,NaN
337,1793-5288,2008,2012,0.0,NaN
403,2055-026X,2015,9999,0.0,NaN
555,0041-1191,1961,2015,0.0,NaN
766,2050-0513,2013,2015,0.0,NaN
772,2056-2799,2015,9999,0.0,NaN
963,1745-3925,2005,2014,1.0,NaN


In [37]:
# Export ISSNs without ISSN-l deduped
ejournals_issn_all_merged_not_issnl_dedup = ejournals_issn_all_merged_not_issnl.sort_values('issn', ascending=False).drop_duplicates('issn').sort_index()
ejournals_issn_all_merged_not_issnl_dedup['issn'].to_csv('data/temp/library/ejournals/ejournals_not_issnl_dedup.csv', sep='\t', encoding='utf-8', index=False)
ejournals_issn_all_merged_not_issnl_dedup

,issn,syear,eyear,is_free,issnl
16,0862-920X,1991,1998,1.0,NaN
337,1793-5288,2008,2012,0.0,NaN
403,2055-026X,2015,9999,0.0,NaN
555,0041-1191,1961,2015,0.0,NaN
772,2056-2799,2015,9999,0.0,NaN
963,1745-3925,2005,2014,1.0,NaN
973,1532-0928,1812,9999,1.0,NaN
1452,2325-6095,2014,9999,0.0,NaN
1582,1538-3318,2002,9999,1.0,NaN
1599,1535-3850,1998,9999,0.0,NaN


In [ ]:
# Concat and dedup ISSNs without ISSN-L to check the problem, some are mistakes or old ISSNs availables also in PubMed metadata
#(we can fix it or send it to the ISSN.org team to improve their data or tell us why they are not matched)
issn_all_not_issnl = ejournals_issn_all_merged_not_issnl_dedup.append(journals_issn_merged_not_issnl_dedup, ignore_index=True)
issn_all_not_issnl_dedup = issn_all_not_issnl.sort_values('issn', ascending=False).drop_duplicates('issn').sort_index()
issn_all_not_issnl_dedup['issn'].to_csv('data/temp/library/journals_not_issnl_dedup.csv', sep='\t', encoding='utf-8', index=False)
issn_all_not_issnl_dedup


In [39]:
# put ISSN at ISSN-L place if ISSN-L is empty
import numpy as np
ejournals_issn_all_merged.issnl.replace(np.NaN, ejournals_issn_all_merged.issn, inplace=True)
ejournals_issn_all_merged[ejournals_issn_all_merged.issnl.isnull()]

,issn,syear,eyear,is_free,issnl


In [40]:
ejournals_issn_all_merged.to_csv('data/temp/library/ejournals/ejournals_ready_to_merge.csv', sep='\t', encoding='utf-8', index=False)

## Merge PubMed metadata with journals coverage

### Import PubMed metadata from XML

The export "VIPID" contains those fields :

 - MedlineCitation/PMID
 - MedlineCitation/Article/ELocationID[@EIdType='doi']
 - PubmedData/ArticleIdList/ArticleId[@IdType='doi']
 - MedlineCitation/Article/Journal/ISSN
 - MedlineCitation/MedlineJournalInfo/ISSNLinking
 - MedlineCitation/Article/Journal/ISOAbbreviation
 - MedlineCitation/Article/Journal/JournalIssue/PubDate/Year
 - MedlineCitation/Article/Journal/JournalIssue/PubDate/MedlineDate
 - MedlineCitation/Article/Journal/JournalIssue/Volume
 - MedlineCitation/Article/Journal/JournalIssue/Issue
 - MedlineCitation/Article/Pagination/MedlinePgn
 - PubmedData/PublicationStatus
 - MedlineCitation/@Status
 
 PubMed data contains **27'837'540** PMIDs (rows)

In [1]:
# Open PubMed data
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
df_pubmed = dd.read_csv('data/sources/pubmed/extractions_xml/extraction_vipid/csv/*.csv', sep='|',
                        names=('pmid', 'doi1', 'doi2', 'issn', 'issnl', 'iso', 'year', 'date', 'vol', 'issue', 'pages', 'status1', 'status2'),
                        dtype={'pmid': 'int', 'issn': 'object', 'issnl': 'object', 'year': 'object', 'date': 'object'},
                        usecols=('pmid', 'year', 'date', 'issn', 'issnl'))
df_pubmed

,pmid,issn,issnl,year,date
npartitions=928,,,,,
,int32,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [2]:
df_pubmed.head()

,pmid,issn,issnl,year,date
0,2,1090-2104,0006-291X,1975,NaN
1,1,0006-2944,0006-2944,1975,NaN
2,3,0006-291X,0006-291X,1975,NaN
3,4,1090-2104,0006-291X,1975,NaN
4,6,0006-291X,0006-291X,1975,NaN


In [3]:
df_pubmed.tail()

,pmid,issn,issnl,year,date
27547,29172298,0004-4849,0004-4849,2016,NaN
27548,29172296,2476-762X,1513-7368,2017,NaN
27549,29172299,0393-974X,0393-974X,2017,NaN
27550,29172297,2476-762X,1513-7368,2017,NaN
27551,29172300,0393-974X,0393-974X,2017,NaN


In [4]:
# fix years empty problem
with ProgressBar():
    df_pubmed_date_vide = df_pubmed.loc[df_pubmed.year.isnull()].compute()
df_pubmed_date_vide['year'] = df_pubmed_date_vide['date'].str.extract('([12][0-9][0-9][0-9])', expand=False)
df_pubmed_date_vide.loc[df_pubmed_date_vide['year'].isnull()]

[########################################] | 100% Completed | 37.0s


,pmid,issn,issnl,year,date
17134,28858454,1532-0650,0002-838X,NaN,8/15/12
814,28872285,0033-2100,0033-2100,NaN,2
819,28872290,0033-2100,0033-2100,NaN,2


In [5]:
# check PMIDs in PubMed and clean the lines by PMID
df_pubmed_date_vide.loc[df_pubmed_date_vide['pmid'] == 28858454, 'year'] = 2012
df_pubmed_date_vide.loc[df_pubmed_date_vide['pmid'] == 28872285, 'year'] = 2017
df_pubmed_date_vide.loc[df_pubmed_date_vide['pmid'] == 28872290, 'year'] = 2017
df_pubmed_date_vide.loc[df_pubmed_date_vide['year'].isnull()]

,pmid,issn,issnl,year,date


In [6]:
# drop na from df_pubmed and append data with fixed years
df_pubmed = df_pubmed.dropna(how='any', subset=['year'])
df_pubmed = df_pubmed.append(df_pubmed_date_vide)
df_pubmed

,pmid,issn,issnl,year,date
npartitions=929,,,,,
,int32,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [7]:
with ProgressBar():
    df_pubmed_pandas = df_pubmed[['pmid', 'issn', 'issnl', 'year']].compute()
df_pubmed_pandas

[########################################] | 100% Completed | 42.5s


,pmid,issn,issnl,year
0,2,1090-2104,0006-291X,1975
1,1,0006-2944,0006-2944,1975
2,3,0006-291X,0006-291X,1975
3,4,1090-2104,0006-291X,1975
4,6,0006-291X,0006-291X,1975
5,7,1873-2968,0006-2952,1975
6,8,1873-2968,0006-2952,1975
7,5,1090-2104,0006-291X,1975
8,9,0006-2952,0006-2952,1975
9,11,0006-2952,0006-2952,1975


In [8]:
# pubmed lines without ISSN-L
df_pubmed_pandas.loc[df_pubmed_pandas['issnl'].isnull()]

,pmid,issn,issnl,year
150,151,NaN,NaN,1975
151,152,NaN,NaN,1975
299,300,NaN,NaN,1975
300,301,NaN,NaN,1975
301,302,NaN,NaN,1975
302,303,NaN,NaN,1975
303,305,NaN,NaN,1975
304,304,NaN,NaN,1975
305,306,NaN,NaN,1975
306,307,NaN,NaN,1975


In [9]:
# pubmed lines without ISSN-L but ISSN
df_pubmed_pandas2 = df_pubmed_pandas.loc[df_pubmed_pandas['issnl'].isnull() & df_pubmed_pandas['issn'].notnull()]
df_pubmed_pandas2

,pmid,issn,issnl,year
659,659,1897-9483,NaN,1975
793,795,0256-9574,NaN,1975
798,796,0256-9574,NaN,1975
1409,1436,0002-7944,NaN,1975
1684,1711,0143-2524,NaN,1976
3594,3771,1897-9483,NaN,1976
3599,3770,1897-9483,NaN,1976
3679,3857,0256-9574,NaN,1976
3684,3858,0256-9574,NaN,1976
3705,3885,0043-0781,NaN,1975


In [10]:
# use ISSN when ISSN-L is empty
del df_pubmed_pandas2['issnl']
df_pubmed_pandas2['issnl'] = df_pubmed_pandas2['issn']
df_pubmed_pandas2

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,pmid,issn,year,issnl
659,659,1897-9483,1975,1897-9483
793,795,0256-9574,1975,0256-9574
798,796,0256-9574,1975,0256-9574
1409,1436,0002-7944,1975,0002-7944
1684,1711,0143-2524,1976,0143-2524
3594,3771,1897-9483,1976,1897-9483
3599,3770,1897-9483,1976,1897-9483
3679,3857,0256-9574,1976,0256-9574
3684,3858,0256-9574,1976,0256-9574
3705,3885,0043-0781,1975,0043-0781


In [11]:
# use the other part and append two parts
df_pubmed_pandas1 = df_pubmed_pandas.loc[~(df_pubmed_pandas['issnl'].isnull() & df_pubmed_pandas['issn'].notnull())]
df_pubmed_pandas1

,pmid,issn,issnl,year
0,2,1090-2104,0006-291X,1975
1,1,0006-2944,0006-2944,1975
2,3,0006-291X,0006-291X,1975
3,4,1090-2104,0006-291X,1975
4,6,0006-291X,0006-291X,1975
5,7,1873-2968,0006-2952,1975
6,8,1873-2968,0006-2952,1975
7,5,1090-2104,0006-291X,1975
8,9,0006-2952,0006-2952,1975
9,11,0006-2952,0006-2952,1975


In [12]:
df_pubmed_end = df_pubmed_pandas1.append(df_pubmed_pandas2, ignore_index=True)
df_pubmed_end

,issn,issnl,pmid,year
0,1090-2104,0006-291X,2,1975
1,0006-2944,0006-2944,1,1975
2,0006-291X,0006-291X,3,1975
3,1090-2104,0006-291X,4,1975
4,0006-291X,0006-291X,6,1975
5,1873-2968,0006-2952,7,1975
6,1873-2968,0006-2952,8,1975
7,1090-2104,0006-291X,5,1975
8,0006-2952,0006-2952,9,1975
9,0006-2952,0006-2952,11,1975


In [13]:
# dedup by PMID
df_pubmed_end_duplicates = df_pubmed_end.loc[df_pubmed_end.duplicated(subset='pmid')]
df_pubmed_end_duplicates

,issn,issnl,pmid,year
19681271,2157-3999,2157-3999,20029611,2009
19681273,2157-3999,2157-3999,20029614,2009
19693729,2157-3999,2157-3999,20029614,2009
19693730,2157-3999,2157-3999,20029614,2009
19693731,2157-3999,2157-3999,20029614,2009
19693732,2157-3999,2157-3999,20029614,2009
19693733,2157-3999,2157-3999,20029614,2009
19693734,2157-3999,2157-3999,20029614,2009
19693735,2157-3999,2157-3999,20029614,2009
19693736,2157-3999,2157-3999,20029614,2009


In [14]:
df_pubmed_end = df_pubmed_end.drop_duplicates(subset='pmid')
df_pubmed_end

,issn,issnl,pmid,year
0,1090-2104,0006-291X,2,1975
1,0006-2944,0006-2944,1,1975
2,0006-291X,0006-291X,3,1975
3,1090-2104,0006-291X,4,1975
4,0006-291X,0006-291X,6,1975
5,1873-2968,0006-2952,7,1975
6,1873-2968,0006-2952,8,1975
7,1090-2104,0006-291X,5,1975
8,0006-2952,0006-2952,9,1975
9,0006-2952,0006-2952,11,1975


In [15]:
# normalize ISSNL
# remove blanks
df_pubmed_end['issnl'] = df_pubmed_end['issnl'].str.strip()
# convert to lower case
df_pubmed_end['issnl'] = df_pubmed_end['issnl'].str.lower()

C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\IRIARTE\AppData\Local\Continuum\Miniconda3\envs\ipykernel_py3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
# export to CSV
df_pubmed_end.to_csv('data/sources/pubmed/pmid_issn_issnl_year_clean.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')

In [1]:
# Check the export
import pandas as pd
pubmed = pd.read_csv('data/sources/pubmed/pmid_issn_issnl_year_clean.csv.gz', delimiter='\t',
                     dtype={'pmid': 'int', 'issn': 'object', 'issnl': 'object', 'year': 'object'}, 
                     header=0)
pubmed

,issn,issnl,pmid,year
0,1090-2104,0006-291x,2,1975
1,0006-2944,0006-2944,1,1975
2,0006-291X,0006-291x,3,1975
3,1090-2104,0006-291x,4,1975
4,0006-291X,0006-291x,6,1975
5,1873-2968,0006-2952,7,1975
6,1873-2968,0006-2952,8,1975
7,1090-2104,0006-291x,5,1975
8,0006-2952,0006-2952,9,1975
9,0006-2952,0006-2952,11,1975


In [2]:
pubmed.dtypes

issn     object
issnl    object
pmid      int32
year     object
dtype: object

In [8]:
# check the data because year is not int
counts = pubmed['year'].value_counts()
counts.to_csv('data/sources/pubmed/year_counts.csv', sep='\t', index=True)

In [13]:
# fixing one wrong line in export file with year = '0021-9258':
# grep '0021-9258' pmid_issn_issnl_year_clean.csv
# [sect ]\        [sect ]\        16040616        0021-9258
# pmid = 16040616, year = 2005 ; issn = 0021-9258 ; issnl = 0021-9258
pubmed.loc[pubmed['year'] == '0021-9258']

,issn,issnl,pmid,year


In [12]:
pubmed.loc[pubmed['pmid'] == 16040616, 'year'] = 2005
pubmed.loc[pubmed['pmid'] == 16040616, 'issn'] = '0021-9258'
pubmed.loc[pubmed['pmid'] == 16040616, 'issnl'] = '0021-9258'
pubmed.loc[pubmed['pmid'] == 16040616]

,issn,issnl,pmid,year
27699540,0021-9258,0021-9258,16040616,2005


In [15]:
# export the fixed dataframe to CSV overwriting the old one
pubmed.to_csv('data/sources/pubmed/pmid_issn_issnl_year_clean.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')

## Merge PubMed and print journals information

In [1]:
# Restart kernel
# Open Pubmed data
import pandas as pd
pubmed = pd.read_csv('data/sources/pubmed/pmid_issn_issnl_year_clean.csv.gz', delimiter='\t',
                     dtype={'pmid': 'int', 'issn': 'object', 'issnl': 'object', 'year': 'int'}, 
                     header=0)
pubmed

,issn,issnl,pmid,year
0,1090-2104,0006-291x,2,1975
1,0006-2944,0006-2944,1,1975
2,0006-291X,0006-291x,3,1975
3,1090-2104,0006-291x,4,1975
4,0006-291X,0006-291x,6,1975
5,1873-2968,0006-2952,7,1975
6,1873-2968,0006-2952,8,1975
7,1090-2104,0006-291x,5,1975
8,0006-2952,0006-2952,9,1975
9,0006-2952,0006-2952,11,1975


In [2]:
# import pjournals data
pjournals = pd.read_csv('data/temp/library/pjournals/pjournals_ready_to_pubmed_merge.csv', delimiter='\t', header=0,
                       dtype={'issn': 'object', 'issnl': 'object', 'syear': 'int', 'eyear': 'int'},
                       usecols=('issnl', 'syear', 'eyear'))
pjournals

,issnl,syear,eyear
0,0003-9772,1966,1976
1,0041-1345,1985,2006
2,0071-7916,1973,1995
3,0079-6824,1961,1973
4,0079-6824,1974,1974
5,0266-7681,1984,1994
6,0266-7681,1995,2012
7,0300-8827,1964,1976
8,0300-8827,1977,9999
9,0362-2436,1982,2009


In [3]:
# normalize ISSNL
# remove blanks
pjournals['issnl'] = pjournals['issnl'].str.strip()
# convert to lower case
pjournals['issnl'] = pjournals['issnl'].str.lower()

In [4]:
# rename years
pjournals = pjournals.rename(columns = {'syear': 'psyear'})
pjournals = pjournals.rename(columns = {'eyear': 'peyear'})

In [5]:
# merge PubMed with pjournals data by ISSNL
pubmed = pubmed.merge(pjournals, on = 'issnl', how='left')
pubmed

,issn,issnl,pmid,year,psyear,peyear
0,1090-2104,0006-291x,2,1975,1961.0,1961.0
1,1090-2104,0006-291x,2,1975,1963.0,2008.0
2,1090-2104,0006-291x,2,1975,1963.0,1993.0
3,1090-2104,0006-291x,2,1975,1960.0,1960.0
4,0006-2944,0006-2944,1,1975,1969.0,1985.0
5,0006-291X,0006-291x,3,1975,1961.0,1961.0
6,0006-291X,0006-291x,3,1975,1963.0,2008.0
7,0006-291X,0006-291x,3,1975,1963.0,1993.0
8,0006-291X,0006-291x,3,1975,1960.0,1960.0
9,1090-2104,0006-291x,4,1975,1961.0,1961.0


In [6]:
# Add PJ column
pubmed['PJ'] = 0
# Calculate if years are in range
pubmed.loc[pubmed['year'] >= pubmed['psyear'], 'PJ'] = 1
pubmed.loc[pubmed['year'] > pubmed['peyear'], 'PJ'] = 0
pubmed

,issn,issnl,pmid,year,psyear,peyear,PJ
0,1090-2104,0006-291x,2,1975,1961.0,1961.0,0
1,1090-2104,0006-291x,2,1975,1963.0,2008.0,1
2,1090-2104,0006-291x,2,1975,1963.0,1993.0,1
3,1090-2104,0006-291x,2,1975,1960.0,1960.0,0
4,0006-2944,0006-2944,1,1975,1969.0,1985.0,1
5,0006-291X,0006-291x,3,1975,1961.0,1961.0,0
6,0006-291X,0006-291x,3,1975,1963.0,2008.0,1
7,0006-291X,0006-291x,3,1975,1963.0,1993.0,1
8,0006-291X,0006-291x,3,1975,1960.0,1960.0,0
9,1090-2104,0006-291x,4,1975,1961.0,1961.0,0


In [8]:
# sort values and drop duplicates to week the good rows (with 1)
pubmed = pubmed.sort_values(by=['PJ', 'pmid'], ascending=[False, True])
pubmed

,issn,issnl,pmid,year,psyear,peyear,PJ
4,0006-2944,0006-2944,1,1975,1969.0,1985.0,1
1,1090-2104,0006-291x,2,1975,1963.0,2008.0,1
2,1090-2104,0006-291x,2,1975,1963.0,1993.0,1
6,0006-291X,0006-291x,3,1975,1963.0,2008.0,1
7,0006-291X,0006-291x,3,1975,1963.0,1993.0,1
10,1090-2104,0006-291x,4,1975,1963.0,2008.0,1
11,1090-2104,0006-291x,4,1975,1963.0,1993.0,1
42,1090-2104,0006-291x,5,1975,1963.0,2008.0,1
43,1090-2104,0006-291x,5,1975,1963.0,1993.0,1
14,0006-291X,0006-291x,6,1975,1963.0,2008.0,1


In [9]:
# drop duplicates
pubmed = pubmed.drop_duplicates(subset='pmid')

In [10]:
pubmed

,issn,issnl,pmid,year,psyear,peyear,PJ
4,0006-2944,0006-2944,1,1975,1969.0,1985.0,1
1,1090-2104,0006-291x,2,1975,1963.0,2008.0,1
6,0006-291X,0006-291x,3,1975,1963.0,2008.0,1
10,1090-2104,0006-291x,4,1975,1963.0,2008.0,1
42,1090-2104,0006-291x,5,1975,1963.0,2008.0,1
14,0006-291X,0006-291x,6,1975,1963.0,2008.0,1
17,1873-2968,0006-2952,7,1975,1958.0,2007.0,1
29,1873-2968,0006-2952,8,1975,1958.0,2007.0,1
45,0006-2952,0006-2952,9,1975,1958.0,2007.0,1
69,1873-2968,0006-2952,10,1975,1958.0,2007.0,1


In [12]:
# PMIDs covered by print journals 
pubmed.loc[pubmed['PJ'] == 1].shape

(9624833, 7)

In [13]:
# export the result
pubmed.to_csv('data/results/pubmed_pj.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')
pubmed[['pmid', 'year', 'PJ']].to_csv('data/results/pubmed_pj_short.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')

## Merge PubMed and e-journals information

In [1]:
# Restart kernel
# Open Pubmed data
import pandas as pd
pubmed = pd.read_csv('data/results/pubmed_pj.csv.gz', delimiter='\t',
                     dtype={'pmid': 'int', 'issn': 'object', 'issnl': 'object', 'year': 'int', 'PJ': 'int'}, 
                     usecols=('pmid', 'issnl', 'year', 'PJ'), header=0)
pubmed

,issnl,pmid,year,PJ
0,0006-2944,1,1975,1
1,0006-291x,2,1975,1
2,0006-291x,3,1975,1
3,0006-291x,4,1975,1
4,0006-291x,5,1975,1
5,0006-291x,6,1975,1
6,0006-2952,7,1975,1
7,0006-2952,8,1975,1
8,0006-2952,9,1975,1
9,0006-2952,10,1975,1


In [2]:
# import ejournals data
ejournals = pd.read_csv('data/temp/library/ejournals/ejournals_ready_to_merge.csv', delimiter='\t', header=0,
                       dtype={'issn': 'object', 'issnl': 'object', 'syear': 'int', 'eyear': 'int', 'is_free': 'float'},
                       usecols=('issnl', 'syear', 'eyear', 'is_free'))
ejournals

,syear,eyear,is_free,issnl
0,1956,2006,1.0,0001-5237
1,1849,9999,1.0,0004-6256
2,1949,9999,1.0,0004-6264
3,1889,9999,1.0,0004-6280
4,1969,2000,1.0,0004-6361
5,1895,1996,1.0,0004-637X
6,1981,2002,1.0,0007-0297
7,1877,2013,1.0,0029-7704
8,1969,1986,1.0,0065-7433
9,1976,1994,1.0,0066-9997


In [3]:
ejournals['is_free'].value_counts()

0.0    66847
1.0    60048
Name: is_free, dtype: int64

In [4]:
# check is_free empty
ejournals.loc[ejournals['is_free'].isnull()]

,syear,eyear,is_free,issnl
20867,999,9999,NaN,1857-9760
35968,2016,9999,NaN,2383-3971
100093,2016,9999,NaN,2383-3971


In [5]:
# replace empty values with 0
ejournals.loc[20867, 'is_free'] = 0
ejournals.loc[35968, 'is_free'] = 0
ejournals.loc[100093, 'is_free'] = 0
ejournals['is_free'] = ejournals['is_free'].astype(int)
ejournals['is_free'].value_counts()

0    66850
1    60048
Name: is_free, dtype: int64

In [6]:
# normalize ISSNL
# remove blanks
ejournals['issnl'] = ejournals['issnl'].str.strip()
# convert to lower case
ejournals['issnl'] = ejournals['issnl'].str.lower()

In [7]:
# rename years
ejournals = ejournals.rename(columns = {'syear': 'esyear'})
ejournals = ejournals.rename(columns = {'eyear': 'eeyear'})

In [8]:
# merge PubMed with ejournals data by ISSNL
pubmed = pubmed.merge(ejournals, on='issnl', how='left')
pubmed

,issnl,pmid,year,PJ,esyear,eeyear,is_free
0,0006-2944,1,1975,1,1967.0,1985.0,0.0
1,0006-2944,1,1975,1,1967.0,1985.0,0.0
2,0006-2944,1,1975,1,1967.0,1985.0,0.0
3,0006-2944,1,1975,1,1967.0,1985.0,0.0
4,0006-291x,2,1975,1,1959.0,9999.0,0.0
5,0006-291x,2,1975,1,1959.0,1994.0,0.0
6,0006-291x,2,1975,1,1959.0,9999.0,0.0
7,0006-291x,2,1975,1,1959.0,1994.0,0.0
8,0006-291x,3,1975,1,1959.0,9999.0,0.0
9,0006-291x,3,1975,1,1959.0,1994.0,0.0


In [9]:
# Add EJ column
pubmed['EJ'] = 0
# Calculate if years are in range
pubmed.loc[pubmed['year'] >= pubmed['esyear'], 'EJ'] = 1
pubmed.loc[pubmed['year'] > pubmed['eeyear'], 'EJ'] = 0
pubmed

,issnl,pmid,year,PJ,esyear,eeyear,is_free,EJ
0,0006-2944,1,1975,1,1967.0,1985.0,0.0,1
1,0006-2944,1,1975,1,1967.0,1985.0,0.0,1
2,0006-2944,1,1975,1,1967.0,1985.0,0.0,1
3,0006-2944,1,1975,1,1967.0,1985.0,0.0,1
4,0006-291x,2,1975,1,1959.0,9999.0,0.0,1
5,0006-291x,2,1975,1,1959.0,1994.0,0.0,1
6,0006-291x,2,1975,1,1959.0,9999.0,0.0,1
7,0006-291x,2,1975,1,1959.0,1994.0,0.0,1
8,0006-291x,3,1975,1,1959.0,9999.0,0.0,1
9,0006-291x,3,1975,1,1959.0,1994.0,0.0,1


In [10]:
# export the result (not enough memory to dedup)
# pubmed.to_csv('data/temp/pubmed_pj_ej_not_deduped.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')
pubmed[['pmid', 'year', 'PJ', 'EJ', 'is_free']].to_csv('data/temp/pubmed_pj_ej_short_not_deuped.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')

In [2]:
# Restart kernel
# Open latest data
import pandas as pd
pubmed = pd.read_csv('data/temp/pubmed_pj_ej_short_not_deuped.csv.gz', delimiter='\t',
                     dtype={'pmid': 'int', 'year': 'int', 'PJ': 'int', 'EJ': 'int', 'is_free': 'float'}, header=0)
pubmed

,pmid,year,PJ,EJ,is_free
0,1,1975,1,1,0.0
1,1,1975,1,1,0.0
2,1,1975,1,1,0.0
3,1,1975,1,1,0.0
4,2,1975,1,1,0.0
5,2,1975,1,1,0.0
6,2,1975,1,1,0.0
7,2,1975,1,1,0.0
8,3,1975,1,1,0.0
9,3,1975,1,1,0.0


In [3]:
# sort values and drop duplicates to keep the good rows (with 1)
pubmed = pubmed.sort_values(by=['pmid', 'EJ'], ascending=[True, False])
pubmed

,pmid,year,PJ,EJ,is_free
0,1,1975,1,1,0.0
1,1,1975,1,1,0.0
2,1,1975,1,1,0.0
3,1,1975,1,1,0.0
4,2,1975,1,1,0.0
5,2,1975,1,1,0.0
6,2,1975,1,1,0.0
7,2,1975,1,1,0.0
8,3,1975,1,1,0.0
9,3,1975,1,1,0.0


In [4]:
# drop duplicates
pubmed = pubmed.drop_duplicates(subset='pmid')
pubmed

,pmid,year,PJ,EJ,is_free
0,1,1975,1,1,0.0
4,2,1975,1,1,0.0
8,3,1975,1,1,0.0
12,4,1975,1,1,0.0
16,5,1975,1,1,0.0
20,6,1975,1,1,0.0
24,7,1975,1,0,0.0
26,8,1975,1,0,0.0
28,9,1975,1,0,0.0
30,10,1975,1,0,0.0


In [5]:
# PMIDs covered by print journals 
pubmed.loc[pubmed['EJ'] == 1].shape

(17259603, 5)

In [6]:
# export the result
pubmed.to_csv('data/results/pubmed_pj_ej.csv.gz', sep='\t', encoding='utf-8', index=False, compression='gzip')